In [ ]:
import re
import pandas as pd
import nltk
import string
import glob
import string
import inflection
import json
import subprocess
from unqlite import UnQLite
import pickle
from scipy import sparse
import re
import numpy as np
from nltk.corpus import stopwords
from collections import Counter
from nltk import wordpunct_tokenize
from nltk.stem.porter import PorterStemmer
from re import finditer
stemmer = PorterStemmer()
from skopt import *
class CorpusParser:

	def __init__(self, filename):
		self.filename = filename
		self.regex = re.compile('^#\s*\d+')
		self.corpus = dict()

	def parse(self):
		with open(self.filename) as f:
			s = ''.join(f.readlines())
		blobs = s.split('#')[1:]
		for x in blobs:
			text = x.split()
			docid = text.pop(0)
			self.corpus[docid] = text

	def get_corpus(self):
		return self.corpus


class QueryParser:

	def __init__(self, filename):
		self.filename = filename
		self.queries = []

	def parse(self):
		with open(self.filename) as f:
			lines = ''.join(f.readlines())
		self.queries = [x.rstrip().split() for x in lines.split('\n')[:-1]]

	def get_queries(self):
		return self.queries


# if __name__ == '__main__':
#     qp = QueryParser('/data/hdj/BM25/text/queries.txt')
#     qp.parse()
#     print (qp.get_queries())
#     cp=CorpusParser('/data/hdj/BM25/text/corpus.txt')
#     cp.parse()
    
    

def _process_notes(note,note_not_tokenize, repository_path):
    #ea6b85284ac8a5b67421bafebe99a7cc2ca6d73e 00001f23f4c6321b7b4468d2eae6b4a1e0e24bce
    (note_content_sha, note_object_sha) = note.split(' ')
    (note_content_sha_not_tokenize, note_object_sha_not_tokenize) = note_not_tokenize.split(' ')

    note_content = cat_file_blob(repository_path, note_content_sha)
    note_content_not_tokenize = cat_file_blob(repository_path, note_content_sha_not_tokenize)
    ast_extraction_result = json.loads(note_content)
    ast_extraction_result_not_tokenize = json.loads(note_content_not_tokenize)
    ast_extraction_result['methodContent']=ast_extraction_result_not_tokenize['methodContent']
    return note_object_sha, ast_extraction_result
def cat_file_blob(repository_path, sha, encoding='latin-1'):
    #git -C /data/hdj/SourceFile/tracking_buggy_files_tomcat_dataset cat-file blob ea6b85284ac8a5b67421bafebe99a7cc2ca6d73e > note_content.txt
    #返回所有文件相关的信息 包括分割好的方法和源码 dict格式
    cmd = ' '.join(['git', '-C', repository_path, 'cat-file', 'blob', sha])
    cat_file_process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    result = cat_file_process.stdout.read().decode(encoding)
    return result

def _f(args):
    return _process_notes(args[0], args[1],args[2])
def list_notes(repository_path, refs='refs/notes/commits'):
    #当ref=='refs/notes/tokenized_counters'时，解析出来的就是tokenized后的文件
    #git -C /data/hdj/SourceFile/tracking_buggy_files_tomcat_dataset notes --ref refs/notes/commits list
    #返回一堆 shas
    cmd = ' '.join(['git', '-C', repository_path, 'notes', '--ref', refs, 'list'])
    notes_lines = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().decode('latin-1').split('\n')
    return notes_lines

tokenized_refs = 'refs/notes/tokenized_counters'
ast_notes = list_notes(repository_path, refs=tokenized_refs)
ast_notes_not_tokenize=list_notes(repository_path)
print("existing tokenized notes ", len(ast_notes))
print("existing not tokenized notes ", len(ast_notes_not_tokenize))

ast_notes[:2],ast_notes_not_tokenize[:2]


note_object_sha=None
ast_extraction_result=None
for note,note_not_tokenize in zip(ast_notes,ast_notes_not_tokenize):
    note_object_sha, ast_extraction_result=_f([note,note_not_tokenize, repository_path])
    break

note_object_sha

# ast_extraction_result


repository_path='/data/hdj/SourceFile/tracking_buggy_files_tomcat_dataset'
# refs='refs/notes/commits'
refs='refs/notes/tokenized_counters'
cmd = ' '.join(['git', '-C', repository_path, 'notes', '--ref', refs, 'list'])
notes_lines = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().decode('latin-1').split('\n')

len(notes_lines)

repository_path='/data/hdj/SourceFile/tracking_buggy_files_tomcat_dataset'
refs_not_tokenize='refs/notes/commits'
cmd = ' '.join(['git', '-C', repository_path, 'notes', '--ref', refs_not_tokenize, 'list'])
notes_lines_not_tokenize = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().decode('latin-1').split('\n')

len(notes_lines_not_tokenize)

sha='52179dced2a1cf65ba73119a8296d362d7d0107e'
encoding='latin-1'
cmd = ' '.join(['git', '-C', repository_path, 'cat-file', 'blob', sha])
cat_file_process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
result = cat_file_process.stdout.read().decode(encoding)

sha_not_tokenize='ea6b85284ac8a5b67421bafebe99a7cc2ca6d73e '
encoding='latin-1'
cmd = ' '.join(['git', '-C', repository_path, 'cat-file', 'blob', sha_not_tokenize])
cat_file_process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
result_not_tokenize = cat_file_process.stdout.read().decode(encoding)

# eval(result_not_tokenize)

# eval(result)

# notes_lines


bug_report_files_collection_db = UnQLite("/data/hdj/tracking_buggy_files/tomcat/tomcat_bug_report_files_collection_db",
                                             flags=0x00000100 | 0x00000001)
tomcat_ast_cache_collection_db=UnQLite("/data/hdj/tracking_buggy_files/tomcat/tomcat_ast_cache_collection_db",
                                             flags=0x00000100 | 0x00000001)

type(bug_report_files_collection_db),type(tomcat_ast_cache_collection_db)

bug_report_files=[item for item in bug_report_files_collection_db]
tomcat_ast=[item for item in tomcat_ast_cache_collection_db]

len(tomcat_ast),len(bug_report_files_collection_db)

# tomcat_ast[0]

# bug_report_files[0]

tomcat_ast_file = pickle.loads(tomcat_ast_cache_collection_db['ebc8fbfa0b2fa6a61fef2955583440192785e6fb'])

type(tomcat_ast_file)
tomcat_ast=[item for item in tomcat_ast_cache_collection_db]
for key,val in tomcat_ast_file.items():
    print(key,type(val),len(val))

len(tomcat_ast_file['tokenizedMethods'])
tomcat_ast_file['tokenizedMethods'][0]

# tomcat_ast_file['methodContent'][:10]

# tomcat_ast_file['tokenizedSource']

# tomcat_ast[:1]

# bug_report_files[:1]

import pickle
current_files = pickle.loads(bug_report_files_collection_db['1cc6641c54f6b0d2dd69ab5323487ffa51f9df9d'])
# bug_report_files_collection_db.close()

type(current_files)
for key,val in current_files.items():
    print(key,type(val),len(val))

# current_files['shas']

# current_files['sha_to_file_name']

# current_files['class_name_to_sha']

# 检查vectorize_ast生成的数据格式

data_prefix='/data/hdj/tracking_buggy_files/tomcat/tomcat'
asts = UnQLite(data_prefix+"_ast_index_collection_index_db", flags = 0x00000100 | 0x00000001)
types = UnQLite(data_prefix+"_ast_types_collection_index_db", flags = 0x00000100 | 0x00000001)
bug_report_index_collection = UnQLite(data_prefix+"_bug_report_index_collection_index_db")
raw_count_data = sparse.load_npz(data_prefix+'_raw_count_data.npz')
feature_names=json.load(open(data_prefix+'_feature_names_dict'))
ast_index=[item for item in asts]
ast_types=[item for item in types]
bug_report_index=[item for item in bug_report_index_collection]

# bug_report_index
raw_count_data.shape
raw_count_data[0]

print(ast_index[0][0],len(ast_index))
print(ast_types[0][0],len(ast_types))
print(bug_report_index[0][0],len(bug_report_index))
ast_index_files = pickle.loads(asts[ast_index[0][0]])
ast_types_files = pickle.loads(types[ast_types[0][0]])
bug_report_index_files = pickle.loads(bug_report_index_collection[bug_report_index[0][0]])

len(feature_names),
# feature_names

print('ast_index_files :',ast_index_files)
print('ast_types_files :',ast_types_files)
print('bug_report_index_files :',bug_report_index_files)

#检查 _partial_enriched_api_index_lookup
bug_report_id='0ae33cb'
feature_names=json.load(open(data_prefix+'_'+bug_report_id+'_partial_enriched_api_index_lookup'))

len(feature_names)

from sklearn.feature_extraction import DictVectorizer

data = [ {'city':'Dubai','temperature':33.},
    {'city':'London','temperature':12.},
    {'city':'San Fransisco','temperature':18.},]
dictVector = DictVectorizer()
vectorize_data=dictVector.fit_transform(data)
print(vectorize_data)
print(dictVector.feature_names_)


type(vectorize_data),vectorize_data

dictVector.inverse_transform(vectorize_data)

from scipy.sparse import coo_matrix, hstack,vstack
a1=coo_matrix([1,1,1,1])
b1=coo_matrix([2,2,2,2])
c1=coo_matrix([3,3,3,3])
rows=[a1,b1,c1]
matrix=sparse.vstack(rows)

matrix.toarray()

# 检查vectorize_ast结束

# 检查convert_tf_idf

raw_count_data = sparse.load_npz(data_prefix+'_0ae2f34_tf_idf_data.npz')
bug_report_id='0ae2f34'
feature_names=json.load(open(data_prefix+'_'+bug_report_id+'_tf_idf_index_lookup'))

print(raw_count_data.shape)
print(len(feature_names))
for i,(key,val) in enumerate(feature_names.items()):
    if i<=1780:
        continue
    print(key,val)

# 检查convert_tf_idf结束

# 检查calculate_feature_3

raw_count_data = sparse.load_npz(data_prefix+'_0ae2f34_feature_3_data.npz')
bug_report_id='0ae2f34'
feature_names=json.load(open(data_prefix+'_'+bug_report_id+'_feature_3_index_lookup'))

# feature_names
python3 fix_and_augment.py /data/hdj/tracking_buggy_files/swt/swt_base.json /data/hdj/SourceFile/tracking_buggy_files_swt_dataset/ > swt_aug.json

print(raw_count_data.shape)
print(len(feature_names))
for i,(key,val) in enumerate(feature_names.items()):
    if i<=1780:
        continue
    print(key,val)

len(sum(raw_count_data.toarray()))
for num in sum(raw_count_data.toarray()):
    if num!=0:
        print(num)

raw_count_data = sparse.load_npz(data_prefix+'_0ae2f34_feature_3_data.npz')

# 检查calculate_feature_3结束

# 检查save_normalized_fold_dataframes.py

file_path = '/data/hdj/tracking_buggy_files/swt/swt_fe06bdb_features.npz'
features_data = sparse.load_npz(file_path).tocsr()

print(features_data.shape)
features_data.toarray()

file_path='/data/hdj/tracking_buggy_files/swt/swt_fe06bdb_files'
filenames=None
with open(file_path, 'r') as f:
    files_list = json.load(f)
#     return files_list
    print(type(files_list))
    filenames=files_list

df = pd.DataFrame(features_data.todense(), index=filenames)
df.columns = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15',
                      'f16', 'f17', 'f18', 'f19', 'used_in_fix']

print(df.shape)
df.head()

swt_normalized_training_fold_0=pd.read_pickle('/data/hdj/tracking_buggy_files/swt/swt_normalized_training_fold_0')

print(swt_normalized_training_fold_0.index.get_level_values(0).unique().shape[0])
print(swt_normalized_training_fold_0.index.get_level_values(1).unique().shape[0])
#虽然每个report都对应200+个文档，但是文档是有重复的，去重后的个数为7186个

index=list(swt_normalized_training_fold_0.index.get_level_values(1).unique())

index=sorted(index)
index[:10],index[7180:]

max_level_0=500
max_level_1=7186
fold=0
#将第一维的索引从0开始重新索引
swt_normalized_training_fold_0.index.set_levels(range(fold * max_level_0, (fold + 1) * max_level_0), level=0, inplace=True)
# swt_normalized_training_fold_0.index.set_levels(range(fold * max_level_1, (fold + 1) * max_level_1), level=1, inplace=True)

swt_normalized_training_fold_0.tail()

swt_normalized_training_fold_0.tail()

swt_normalized_training_fold_0.index.set_levels(range(fold * max_level_1, (fold + 1) * max_level_1), level=1, inplace=True)

swt_normalized_training_fold_0.loc['1839d9b'].shape

print(swt_normalized_training_fold_0.shape)
swt_normalized_training_fold_0.head()

swt_normalized_training_fold_0[swt_normalized_training_fold_0['used_in_fix']==1].shape

type(swt_normalized_training_fold_0.loc['1839d9b'])

swt_normalized_training_fold_0.loc['b067e67']

swt_normalized_training_fold_0.shape
swt_normalized_training_fold_0[swt_normalized_training_fold_0['used_in_fix']==1]
swt_normalized_training_fold_0.head(10)

swt_normalized_training_fold_0.tail()

df[df['used_in_fix']==1]

# 检查save_normalized_fold_dataframes.py结束

src_addresses = glob.glob(str('/data/hdj/tracking_buggy_files/swt') + '/**/*.npz', recursive=True)

len(src_addresses),len(size)
size1=[v[0] for v in size]

max(size1),min(size1),sum(size1),len(src_addresses),sum(size1)/len(src_addresses)

raw_count_data.toarray()

size=[]
for file in src_addresses:
    raw_count_data = sparse.load_npz(file)
    print(raw_count_data.shape)
    size.append(raw_count_data.shape)
    break


data_filename_memmap='/data/hdj/tracking_buggy_files/joblib_memmap_tomcat/tomcat/data_memmap'
fold_number, fold_testing, fold_training = load(data_filename_memmap, mmap_mode="r")

fold_number,type(fold_testing),type(fold_training)

for i,(key ,val) in enumerate(fold_training.items()):
    if i<=1:
#         break
        continue
    print(key)
    print(type(val),val.shape)
    print(val)

for i,(key ,val) in enumerate(fold_testing.items()):
    if i<=1:
#         break
        continue
    print(key)
    print(type(val),val.shape)
    print(val)

print(cp.get_corpus()['2'])

corpus=cp.get_corpus()

for i,(key,val) in enumerate(corpus.items()):
    print(key,val)
    if i==2:
        break

#将数据组织成dict格式
#query {bug_id:{'summary':string,'description':string,'fixed_files':string}}
#code {code_path:string}
query=dict()

with open('/data/hdj/SourceFile/data/sourceFile_aspectj/bugreport.txt','r',encoding='utf-8') as f_in:
    for i, line in enumerate(f_in) :
        if i==0:
            continue
#         print(line.strip().split('\t'))
        contents=line.strip().split('\t')
        bug_id=contents[1]
        summary=contents[2]
        description=contents[3]
        #处理files，添加commit
        fixed_list=[]
        for file_path in contents[9].split(' '):
            if ':' in file_path:
                continue
#             str_list=file_path.split('/')
#             str_list[-1]=contents[7]+' '+str_list[-1]
# #             print(str_list)
#             new_string='/'.join(str_list)
#             fixed_list.append(new_string)
            fixed_list.append(file_path)
        query[bug_id]={'summary':summary,'description':description,'fixed_files':fixed_list}
#         print(query)
#         break

# query

def _split_camelcase(tokens):

    # Copy tokens
    returning_tokens = tokens[:]
    for token in tokens:
        split_tokens = re.split(fr'[{string.punctuation}]+', token)
        # If token is split into some other tokens
        if len(split_tokens) > 1:
            returning_tokens.remove(token)
            # Camel case detection for new tokens
            for st in split_tokens:
                camel_split = inflection.underscore(st).split('_')
                if len(camel_split) > 1:
                    returning_tokens.append(st)
                    returning_tokens += camel_split
                else:
                    returning_tokens.append(st)
        else:
            camel_split = inflection.underscore(token).split('_')
            if len(camel_split) > 1:
                returning_tokens += camel_split

    return returning_tokens



split_tokens = re.split(fr'[{string.punctuation}]+', 'java.lang.VerifyError')

split_tokens



query['251326']

#准备code
aspectj_test=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_test_pos_neg.csv')
aspectj_test.head()

#queries准备完成
queries=[]
bugid_set=set()
for bug_id in aspectj_test['id'].values:
#     print(key,val)
    if bug_id in bugid_set:
        continue
#     if bug_id==256400:
    bugid_set.add(bug_id)
    summary=query[str(bug_id)]['summary']
    summary_list=_split_camelcase(nltk.wordpunct_tokenize(summary))
    summary_list = [x for x in summary_list if x != '']
    queries.append(summary_list)

#         break
# print(queries[:2])
# print(queries)

len(queries)

aspectj_root_path='/data/hdj/SourceFile/tracking_buggy_files_aspectj_dataset/'
# path_set=set()
# for path in aspectj_test['path'].values:
#     path_set.add(path)

def code_pre(outstring):
    #主要目的是去除所有的注释 ，import， 替换回车换行符 ，过滤掉词长度小于3，最终返回数据格式是list
    m = re.compile(r'/\*.*?\*/', re.S)
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m = re.compile(r'//.*')
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m=re.compile(r'import.*')
    outtmp=re.sub(m,'',outstring)
    outstring = outtmp
    m = re.compile(r'package.*')
    outtmp = re.sub(m, '', outstring)
    outstring = outtmp
    m = re.compile(r'#.*')
    outtmp = re.sub(m, ' ', outstring)
    outstring = outtmp
    for char in ['\r\n', '\r', '\n']:
        outstring = outstring.replace(char, ' ')
    list_str=outstring.split()
    list_str=[t for t in list_str if len(t)>=3]
    # outstring = ' '.join(list_str)
    return list_str

src_addresses = glob.glob(str('/data/hdj/SourceFile/tracking_buggy_files_aspectj_dataset') + '/**/*.java', recursive=True)
len(src_addresses)


reg='/data/hdj/SourceFile/tracking_buggy_files_aspectj_dataset/'
'/data/hdj/SourceFile/tracking_buggy_files_aspectj_dataset/weaver/testsrc/org/aspectj/weaver/CommonsTraceTest.java'.split(reg)

path_set=set()
reg='/data/hdj/SourceFile/tracking_buggy_files_aspectj_dataset/'
for path in src_addresses:
    path_set.add(path.split(reg)[1])
#     src_addresses[:10]

# len(path_set.)
# 'asm/src/org/aspectj/asm/internal/JDTLikeHandleProvider.java' in path_set
path_filter=[]
for i,path in enumerate(path_set):
    path_filter.append(path)
    if i==1000:
        break

len(path_set)
# 'asm/src/org/aspectj/asm/internal/JDTLikeHandleProvider.java' in path_filter

# len(path_set)
# path_set=aspectj_test_pos_neg['path'][1000:1000+1000]

# path_set
corpus=dict()
for path in path_set:
#     print(path)
    code_path=aspectj_root_path+path
    rf = open(code_path, 'r', encoding='utf-8', errors='ignore')
    data = rf.read()
    code = ' '.join(code_pre(data))
#     print(code)
    code_list=_split_camelcase(nltk.wordpunct_tokenize(code))
    code_list = [x for x in code_list if x != '']
#     print(code_list)
    corpus[path]=code_list
#     break

len(corpus),type(corpus)

# for i,(key,val) in enumerate(corpus.items()):
# #     if i==10:
# #         break
#     print(key)

import pickle
with open('/data/hdj/SourceFile/data/sourceFile_eclipseUI/corpus.pickle', 'wb') as file:
        pickle.dump(corpus, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('/data/hdj/SourceFile/data/sourceFile_eclipseUI/querys.pickle', 'wb') as file:
        pickle.dump(queries, file, protocol=pickle.HIGHEST_PROTOCOL)

token=['Bug', '423257', 'LTW', 'java', 'lang', 'VerifyError', 'Bad', 'return', 'type', 'with', 'generics', 'and', 'local', 'variables', '', '', '', '', '', 'verify', 'error', '', '']
token.remove('')



all_results_df_before[:5]

# 检查指标的计算过程，最终找出前50个文件，获得bid fid 与db之间关系

import pickle
# all_results_df_after=pickle.load(open('/data/hdj/tracking_buggy_files/joblib_memmap_swt/all_results_df_after.pickle', 'rb'))
all_results_df_before=pickle.load(open('/data/hdj/tracking_buggy_files/joblib_memmap_swt/all_results_df_before.pickle', 'rb'))

# 生成训练数据 pos vs neg

194+5713,194/5713

#
import pandas as pd
from collections import defaultdict
k=10

train_fold = defaultdict(list)
for i,(bug_report, bug_report_files_dataframe)  in enumerate(all_results_df_before.groupby(level=0, sort=False)):
    sorted_df = bug_report_files_dataframe.sort_values(ascending=False, by=['result'])
# #     print(sorted_df)
#     used_in_fix=sorted_df[sorted_df['used_in_fix']==1.0]
#     not_used_in_fix=sorted_df[sorted_df['used_in_fix']==0.0]
# #     print(used_in_fix)
#     used_in_fix_k=used_in_fix.shape[0]
# #     not_used_in_fix_k=k-used_in_fix_k
#     not_used_in_fix_k=used_in_fix_k
# #     if not_used_in_fix_k<=0:
# #         not_used_in_fix_k=used_in_fix_k
# #     print(used_in_fix.shape)
#     train=pd.concat([used_in_fix,not_used_in_fix[-not_used_in_fix_k:],not_used_in_fix[:not_used_in_fix_k*2]])
#     print(train.shape)
    train=sorted_df[:50]
    train_fold[i//500].append(train)
#     print(len(train_fold[i//500]))
#     if i==2:
#         break

# train_fold[0][:3]

print(len(train_fold))
for key,val in train_fold.items():
    print(key,len(val))
    data=pd.concat(train_fold[key])
    data.to_csv('/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_hard_'+str(key+1)+'.csv',index=True)

train_1=pd.read_csv('/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv')
train_1.head(10)

train_1['used_in_fix'].sum()/train_1.shape[0],train_1['used_in_fix'].sum(),train_1.shape[0]

# 生成训练数据 pos vs neg 1:3结束

# 生成top-50测试数据

#
k=50

train_fold = defaultdict(list)
for i,(bug_report, bug_report_files_dataframe)  in enumerate(all_results_df_before.groupby(level=0, sort=False)):
    sorted_df = bug_report_files_dataframe.sort_values(ascending=False, by=['result'])

    train_fold[i//500].append(sorted_df[:50])
    print(len(train_fold[i//500]))
#     if i==2:
#         break
print(len(train_fold))
for key,val in train_fold.items():
    print(key,len(val))
    data=pd.concat(train_fold[key])
    data.to_csv('/data/hdj/tracking_buggy_files/joblib_memmap_swt/test_'+str(key+1)+'.csv',index=True)

# #最后平均得分 top10rank, MRRrank, MAPrank : 0.212 0.08386737159603685 0.08130901352903222 用的tracking数据预训练的模型
# #最后平均得分 top10rank, MRRrank, MAPrank : 0.308 0.10942885958858604 0.09805982264520696 用的之前预训练保存的模型
# #最后平均得分 top10rank, MRRrank, MAPrank : 0.342 0.14929821137611463 0.1260024167896456
with open('/data/hdj/tracking_buggy_files/joblib_memmap_swt/0521_sumDes_tokSour_50_rawmodel.txt','r',encoding='utf-8') as f_in:
    for i,line in enumerate(f_in):
#         print(line)
        contents=line.split('<CODESPLIT>')
        print(contents[0],contents[1],contents[5],contents[6])
        if i==5:
            break

# 生成top-50测试数据结束

data.to_csv('/data/hdj/tracking_buggy_files/joblib_memmap_swt/test.csv')

test=pd.read_csv('/data/hdj/tracking_buggy_files/joblib_memmap_swt/test.csv')
test.head()

# 计算前k个的recall，将前k个文件保存起来开始

sum_val=0
average_precision_per_bug_report = []
reciprocal_ranks = []
k_range=range(1, 21)
bug_report_number = 0
accuracy_at_k = dict.fromkeys(k_range, 0)
top_k_list=[]
all_pos_list=[]
recall_list=[]
all_results_df_top_k=[]
k=50
left_len=[]
for i,(bug_report, bug_report_files_dataframe)  in enumerate(all_results_df_before.groupby(level=0, sort=False)):
    print('index :',i,bug_report,bug_report_files_dataframe.shape[0])
    sum_val+=bug_report_files_dataframe.shape[0]
#     print(bug_report_files_dataframe)
#     min_fix_result = bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix'] == 1.0]['result'].min()
#     bug_report_files_dataframe2 = bug_report_files_dataframe[bug_report_files_dataframe["result"] >= min_fix_result]
#     print('取fix得分最小值之前的所有文件 ',bug_report_files_dataframe2.shape,' 过滤前大小 :',bug_report_files_dataframe.shape,
#           ' min_fix_result :',min_fix_result)
#     sorted_df = bug_report_files_dataframe2.sort_values(ascending=False, by=['result'])
    sorted_df = bug_report_files_dataframe.sort_values(ascending=False, by=['result'])
    #假如过滤后没有值，直接用原始的dataframe
    if sorted_df.shape[0] == 0:
        sorted_df = bug_report_files_dataframe.copy().sort_values(ascending=False, by=['result'])
#     print(sorted_df)
    top_k=sorted_df[:k]
    top_k_pos=top_k['used_in_fix'].sum()
    all_pos=sorted_df['used_in_fix'].sum()
    recall=top_k_pos/all_pos
    print('top_k :',top_k_pos,' all :',all_pos,' recall :',recall)
    top_k_list.append(top_k_pos)
    all_pos_list.append(all_pos)
    recall_list.append(recall)
    all_results_df_top_k.append(top_k)
    left_len.append(top_k.shape[0])
#     if i==10:
#         break
sum(recall_list)/len(recall_list),len(recall_list)
#10 (0.6132802407120562, 3645)
#20 (0.7098751551130549, 3645)
#30 (0.7619124531045822, 3645)
#50 (0.8176689635982688, 3645)
#60 (0.8375844719269108, 3645)
#70 (0.8500262484623857, 3645)
#80 (0.8607365838268616, 3645)
#90 (0.8704146788592242, 3645)
#100 (0.8796385848682644, 3645
#150 (0.9100992868124254, 3645)
#200 (0.9261051897470557, 3645)
#300 (0.9458304773765746, 3645)

# sorted(left_len,reverse=True)
data=pd.DataFrame()
data['len']=left_len
data['len'].value_counts()
# for key,val in data['len'].value_counts().items():当输出内容太多被折叠时，自己手动打印值
#     print(key,val)

all_results_df_top_k = pd.concat(all_results_df_top_k)

print(all_results_df_top_k.loc['a0c9251'].shape)
all_results_df_top_k

182250/50

# all_results_df_test_fold=dict()
# rows=0
# file_prefix='/data/hdj/tracking_buggy_files/joblib_memmap_swt/swt'
# for i in range(8):
#     all_results_df_test_fold[i+1]=all_results_df_top_k.loc[(i+1)*500:(i+2)*500-1]
#     all_results_df_test_fold[i+1].to_pickle(file_prefix + '_second_testing_fold_' + str(i+1))
#     rows+=all_results_df_test_fold[i+1].shape[0]

rows,all_results_df_top_k.shape

# 根据bid 和fid获取相应的字符

# fold_number, fold_testing, fold_training = load('/data/hdj/tracking_buggy_files/joblib_memmap_swt/swt/data_memmap', mmap_mode="r")

raw=swt_testing_fold_1.loc['a0c9251']

reset=fold_testing[1].loc[500]

# fold_testing[1]

#获取bug_report TODO
def load_bug_reports(bug_report_file_path):
    """load bug report file (the one generated from xml)"""
    with open(bug_report_file_path) as bug_report_file:
        bug_reports = json.load(bug_report_file)
        return bug_reports
bug_report_file_path='/data/hdj/tracking_buggy_files/swt.json'

bug_reports = load_bug_reports(bug_report_file_path)
#这个是原始的report ，我还可以获取经过分词的report
bug_report_id='bae8b23'
current_bug_report = bug_reports[bug_report_id]['bug_report']
current_bug_report

# English stop words
stop_words = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these',
'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but',
'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with',
'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm',
'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn',
'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn',
'weren', 'won', 'wouldn', 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'k', 'l', 'n', 'p',
'q', 'u', 'v', 'w', 'x', 'z', 'us'])

# Java language keywords
java_keywords = set(['abstract', 'assert', 'boolean', 'break', 'byte', 'case',
'catch', 'char', 'class', 'const', 'continue', 'default', 'do', 'double',
'else', 'enum', 'extends', 'false', 'final', 'finally', 'float', 'for', 'goto',
'if', 'implements', 'import', 'instanceof', 'int', 'interface', 'long',
'native', 'new', 'null', 'package', 'private', 'protected', 'public', 'return',
'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this',
'throw', 'throws', 'transient', 'true', 'try', 'void', 'volatile', 'while'])
def clean_string_report(string):
    outtmp=re.findall(r'[\w"]+|[.,!?;{}:()\+\-\*\/=><"]',string)
    outtmp=[token for token in outtmp if token not in stop_words]
    outtmp= [token for token in outtmp if token not in java_keywords]
    outtmp=[token for token in outtmp if token.isalnum() ]
    return ' '.join(outtmp)
clean_string_report(current_bug_report['summary'])
clean_string_report(current_bug_report['description'])





3 0.57

outstring=current_bug_report['description']
for char in ['\r\n', '\r', '\n']:
        outstring = outstring.replace(char, ' ')

#获取code信息
# data_prefix='/data/hdj/tracking_buggy_files/tomcat/tomcat'
# ast_cache_db = UnQLite(data_prefix+"_ast_cache_collection_db")
swt_ast_cache_collection_db=UnQLite("/data/hdj/tracking_buggy_files/swt/swt_ast_cache_collection_db",
                                             flags=0x00000100 | 0x00000001)
# swt_ast=[item for item in swt_ast_cache_collection_db]

# for key,val in tomcat_ast_file.items():
#     print(key,type(val),len(val))

# swt_ast[0]
swt_ast_file = pickle.loads(swt_ast_cache_collection_db['13141282f571747cbdb288e7b94787736c724819'])
# swt_ast_file
for key,val in swt_ast_file.items():
    print(key,type(val),len(val))


# swt_ast_file





# method_top_k

# 生成PLI需要的数据 采用set去重 只取6个tokenized内容

def camel_case_split(identifier):
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]
def mergeTokenziedSource(tokenizedSource):#dict 类型
    return ' '.join(tokenizedSource.keys())
removed = u'!"#%&\'()*+,-./:;<=>?@[\]^_`{|}~1234567890'
utf_translate_table = dict((ord(char), u' ') for char in removed)
stop_words = set(stopwords.words('english'))

def tokenize(text, stemmer):
    sanitized_text = text.translate(utf_translate_table)
    tokens = wordpunct_tokenize(sanitized_text)
    all_tokens = []
    for token in tokens:
        additional_tokens = camel_case_split(token)
        if len(additional_tokens)>1:
            for additional_token in additional_tokens:
                all_tokens.append(additional_token)
        all_tokens.append(token)
    return Counter([stemmer.stem(token) for token in all_tokens if token.lower() not in stop_words])
def convert_dict2string_set(dict_list):
    counter_list = set()
    for dict_item in dict_list:
        for k,v in dict_item.items():
            counter_list.add(k)
#     print('len dict_set :',len(dict_list),'merge len :',len(counter_list))
    return ' '.join(counter_list)
def convert_dict2string_list(dict_list):
    counter_list = []
    for dict_item in dict_list:
        for k,v in dict_item.items():
            counter_list.append(k)
#     print('len dict_list :',len(dict_list),'merge len :',len(counter_list))
    return ' '.join(counter_list)  
def clean_string(string):
    m = re.compile(r'/\*.*?\*/', re.S)
    outstring = re.sub(m, '', string)
    m = re.compile(r'//.*')
    outtmp = re.sub(m, '', outstring)
    for char in ['\r\n', '\r', '\n']:
        outtmp = outtmp.replace(char, ' ')
    outtmp=' '.join(outtmp.split())
    outtmp=re.findall(r'[\w"]+|[.,!?;{}:()\+\-\*\/=><"]',outtmp)
    return ' '.join(outtmp)
def get_method_top_k(choosed_methods:list,k=50):
    method_list=[]
    for string_bef in choosed_methods:
#         print('string_bef :',string_bef)
        string=clean_string(string_bef)
        if '{'  in string and '}'in string :
            method_list.append(string)
#     print('函数体的方法数量: ',len(method_list))
    num=len(method_list)
    method_list = sorted(method_list,key = lambda string:len(string.split()),reverse=True)
#     print(len(method_list))
#     for method in method_list:
#         print(method)
    return method_list[:k],num
method_top_k,num=get_method_top_k(swt_ast_file['methodContent'],10)  
num


train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv' 
train_k=pd.read_csv(train_k_path)
train_k.set_index(['bid','fid'], inplace = True)
train_k.head(10)
all_ast_index=list(train_k.index.get_level_values(1).unique())
method_len=[]
for ast_index in all_ast_index:
#     print(ast_index)
    ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
    print('method len',len(ast_file['methodContent']))
    method_top_k=get_method_top_k(ast_file['methodContent'],10)  
    print('method_top_k :',len(method_top_k))
    method_len.append(len(method_top_k))
    

statis(method_len,'method_len')

# with open('/data/hdj/BERT_PLI/train_1_tokenzied2_15_stage2.json','r',encoding='utf-8') as f_in:
#     for i,line in enumerate(f_in):
#         line=eval(line)
#         print(len(line['c_paras']))
#         if i==100:
#             break
# for ast_index in all_ast_index:
#     print(ast_index)
#     ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
#     print(ast_file['methodContent'])
#     res=get_method_top_k(ast_file['methodContent'],10)
#     print(len(res))
# swt_ast_cache_collection_db


def generate_pli(train_k_path,out_path):
    #生成PLI需要的数据集
    task1_list=[]
    #train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv' 
    #out_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.txt'
    train_k=pd.read_csv(train_k_path)
    train_k.set_index(['bid','fid'], inplace = True)
    train_k.head(10)
    #取bid所对应的report
    bid_list=list(train_k.index.get_level_values(0).unique())
    len(bid_list)
    summarys=[]
    descriptions=[]
    for bug_report_id in bid_list:
        current_bug_report = bug_reports[bug_report_id]['bug_report']
        summarys.append(current_bug_report['summary'])
        descriptions.append(current_bug_report['description'])
    report_dataFrame=pd.DataFrame({'summary':summarys,'description':descriptions},index=bid_list)
    report_dataFrame.index.names=['bid']
    def remove_twoHeadWord(string):
        contents=string.split(' ')[2:]
        return ' '.join(contents)
    report_dataFrame.fillna("",inplace=True)
    report_dataFrame['summary']=report_dataFrame['summary'].apply(remove_twoHeadWord)
    report_dataFrame['summary']=report_dataFrame['summary'].apply(clean_string_report)
    report_dataFrame['description']=report_dataFrame['description'].apply(clean_string_report)
    report_dataFrame.head()
    all_ast_index=list(train_k.index.get_level_values(1).unique())
    type(all_ast_index)
#     all_ast_file_tokenizedSource=[]
    all_ast_file_tokenizedMethods=[]
    all_ast_file_tokenizedClassNames =[]
    all_ast_file_tokenizedMethodNames=[]
    all_ast_file_tokenizedVariableNames=[]
    all_ast_file_tokenizedComments =[]
    all_ast_file_methods=[]
    for ast_index in all_ast_index:
    #     print(ast_index)
        ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
#         all_ast_file_tokenizedSource.append(convert_dict2string_set([ast_file['tokenizedSource']]))
        all_ast_file_tokenizedMethods.append(convert_dict2string_set(ast_file['tokenizedMethods']))
        top_k_method,num=get_method_top_k(ast_file['methodContent'],10)
        print('total method num :',len(ast_file['methodContent']),' 有方法体的函数数量 :',num,' choosed method num :',len(top_k_method))
        all_ast_file_methods.append(top_k_method )
        all_ast_file_tokenizedClassNames.append(convert_dict2string_set(ast_file['tokenizedClassNames']))
        all_ast_file_tokenizedMethodNames.append(convert_dict2string_set(ast_file['tokenizedMethodNames']))
        all_ast_file_tokenizedVariableNames.append(convert_dict2string_set(ast_file['tokenizedVariableNames']))
        all_ast_file_tokenizedComments.append(convert_dict2string_set(ast_file['tokenizedComments']))
    # all_ast_index
    all_ast_index_dataframe=pd.DataFrame({'all_ast_file_methods':all_ast_file_methods,'tokenizedMethods':all_ast_file_tokenizedMethods,
                                          'tokenizedClassNames':all_ast_file_tokenizedClassNames,'tokenizedMethodNames':all_ast_file_tokenizedMethodNames,
                                          'tokenizedVariableNames':all_ast_file_tokenizedVariableNames,'tokenizedComments':all_ast_file_tokenizedComments},index=all_ast_index)
    # print(all_ast_index_dataframe.head())
    print(all_ast_index_dataframe.shape)
    # all_ast_index_dataframe.set_index(["ast_index"], inplace=True)
    # all_ast_index_dataframe
    all_ast_index_dataframe.index.names=['fid']
    all_ast_index_dataframe.head(2)
    #开始连接字段
    all_methodContent_index_dataframe=all_ast_index_dataframe.join(train_k,how='inner')
    all_methodContent_index_dataframe.head()
    all_report_methodContent_index_dataframe=report_dataFrame.join(all_methodContent_index_dataframe,how='inner')
    all_report_methodContent_index_dataframe.head()
    
    task1_list=[]
    all_report_methodContent_index_dataframe.fillna("",inplace=True)
    sum_label=0
    for i,(index, row) in enumerate(all_report_methodContent_index_dataframe.iterrows()):
    #     print(index[0],index[1],row['summary'],row['description'],row['methodContent'],row['used_in_fix'])
#         if len(row['tokenizedSource'])==0:
#             continue
        guid='_'.join([index[0],index[1]])
        
#         q_paras=[mergeTokenziedSource(tokenize(row['summary'],stemmer)),mergeTokenziedSource(tokenize(row['description'],stemmer))]
        q_paras=[row['summary'],row['description']]
        c_paras=[row['tokenizedMethods'],row['tokenizedClassNames'],
                 row['tokenizedMethodNames'],row['tokenizedVariableNames'],row['tokenizedComments'],]
#         print('all_ast_file_methods :',len(row['all_ast_file_methods']))
        c_paras.extend(row['all_ast_file_methods'])
#         print(i,'summary :',type(row['summary']),'description :',type(row['description']),'len summary :',len(q_paras.split()),'source :',len(c_paras.split()))
        summary_len.append(len(q_paras[0].split()))
        description_len.append(len(q_paras[1].split()))
#         tokenizedSource_len.append(len(c_paras[0].split()))

#         tokenizedMethods_len.append(len(c_paras[1].split()))      
#         tokenizedClassNames_len.append(len(c_paras[2].split()))   
#         tokenizedMethodNames_len.append(len(c_paras[3].split()))   
#         tokenizedVariableNames_len.append(len(c_paras[4].split())) 
#         tokenizedComments_len.append(len(c_paras[5].split()))
        label=int(row['used_in_fix'])
        task1=dict()
        task1['guid']=guid
        task1['q_paras']=q_paras
        task1['c_paras']=c_paras
        task1['label']=label
        task1_list.append(task1)
#         if i==10:
#             break
    print(task1_list[0])
    with open(out_path,'w',encoding='utf-8') as f_out:
        for task1 in task1_list:
            out_line = json.dumps(task1, ensure_ascii=False) + '\n'
            f_out.write(out_line)
   
summary_len=[]
description_len=[]
tokenizedSource_len=[]
tokenizedMethods_len=[]
tokenizedClassNames_len=[]
tokenizedMethodNames_len=[]
tokenizedVariableNames_len=[]
tokenizedComments_len=[]
train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv' 
train_out_path='/data/hdj/BERT_PLI/train_1_tokenzied2_15_stage2.json' 
# generate_pli(train_k_path,train_out_path)

test_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/test_2.csv' 
test_out_path='/data/hdj/BERT_PLI/test_2_tokenzied2_15_stage2.json'  
generate_pli(test_k_path,test_out_path)

# summary_len=[]
def statis(sourceLen,name):#list
    a = np.array(sourceLen)
    print(name,'code 的 10%', np.percentile(a, 10))
    print('20 :', np.percentile(a, 20))
    print('30 :', np.percentile(a, 30))
    print('40 :', np.percentile(a, 40))
    print('50 :', np.median(a))
    print('60 :', np.percentile(a, 60))
    print('70 :', np.percentile(a, 70))
    print('80 :', np.percentile(a, 80))
    print('90 :', np.percentile(a, 90))
statis(summary_len,'summary_len')
statis(description_len,'description_len')
statis(tokenizedSource_len,'tokenizedSource_len')
statis(tokenizedMethods_len,'tokenizedMethods_len')
statis(tokenizedClassNames_len,'tokenizedClassNames_len')
statis(tokenizedMethodNames_len,'tokenizedMethodNames_len')
statis(tokenizedVariableNames_len,'tokenizedVariableNames_len')
statis(tokenizedComments_len,'tokenizedComments_len')




# convert_dict2string_list([swt_ast_file['tokenizedSource']])

source

# methods=convert_dict2string_list(swt_ast_file['tokenizedMethods'])
# methods

# methods=convert_dict2string_set(swt_ast_file['tokenizedMethods'])
# methods

ClassNames=convert_dict2string(swt_ast_file['tokenizedClassNames'])
ClassNames

# MethodNames=convert_dict2string_list(swt_ast_file['tokenizedMethodNames'])
# MethodNames

# convert_dict2string_set(swt_ast_file['tokenizedMethodNames'])

# VariableNames=convert_dict2string(swt_ast_file['tokenizedVariableNames'])
# VariableNames


# Comments=convert_dict2string_list(swt_ast_file['tokenizedComments'])
# Comments


# swt_ast_file['tokenizedComments']

convert_dict2string_list(swt_ast_file['tokenizedComments'])

swt_ast_file['tokenizedClassNames'],swt_ast_file['classNames'],

# swt_ast_file['tokenizedSource']

# summary=0
# for method in swt_ast_file['tokenizedMethods']:
# #     print(len(method))
#     summary+=len(method)
# print(summary)





import re
import pandas as pd
import nltk
import string
import glob
import string
import inflection
import json
import subprocess
from unqlite import UnQLite
import pickle
from scipy import sparse
import numpy as np
from skopt import *
import torch

import pandas as pd

import pandas as pd
# swt_testing_fold_1=pd.read_pickle('/data/hdj/tracking_buggy_files/swt/swt_testing_fold_0')
swt_all_results_df_before=pd.read_pickle('/data/hdj/tracking_buggy_files/joblib_memmap_swt/all_results_df_before.pickle')
# # swt_all_results_df_before
# swt_testing_fold_1

swt_all_results_df_before.head(10)

with open('/data/hdj/BERT_PLI/train_1_tokenzied2_6_stage2.json','r',encoding='utf-8') as f_in:
    with open('/data/hdj/BERT_PLI/train_1_tokenzied2_6_stage2_replace.json','w',encoding='utf-8') as f_out:
        for line in f_in:
            line=eval(line)
#             out=dict()
#             out['guid']=line['guid']
#             out['q_paras']=line['text_a']
#             out['c_paras']=line['text_b']
#             out['label']=line['label']
            if line['label']==1 or line['label']=='1':
                for i in range(10):
                    out_line = json.dumps(line, ensure_ascii=False) + '\n'
                    f_out.write(out_line)
            else:
                out_line = json.dumps(line, ensure_ascii=False) + '\n'
                f_out.write(out_line)

# for id in stage2_result.index.unique():
#     print(id)

# count=0
# for id in stage2_result.index.unique():
#     bug_report_files_dataframe=stage2_result.loc[id]
#     min_fix_result = bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix'] == 1.0]['result'].min()
#     bug_report_files_dataframe2 = bug_report_files_dataframe[bug_report_files_dataframe["result"] >= min_fix_result]
#     sorted_df = bug_report_files_dataframe2.sort_values(ascending=False, by=['result'])
#     print(sorted_df.shape)
#     if sorted_df.shape[0]<=20:
#         count+=1
# print(count)

168/500

k_range=range(1, 21)
average_precision_per_bug_report = []
reciprocal_ranks = []
# calculate per each query (bug report)
accuracy_at_k = dict.fromkeys(k_range, 0)
bug_report_number = 0
for bug_report, bug_report_files_dataframe in stage2_result.groupby(level=0, sort=False):
        min_fix_result = bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix'] == 1.0]['result'].min()
        bug_report_files_dataframe2 = bug_report_files_dataframe[bug_report_files_dataframe["result"] >= min_fix_result]
        sorted_df = bug_report_files_dataframe2.sort_values(ascending=False, by=['result'])
#         print(sorted_df)
        precision_at_k = []
        # precision per k in range
        tmp = sorted_df
        a = range(1, tmp.shape[0] + 1)
        tmp['position'] = pd.Series(a, index=tmp.index)
        precision_at_k = []
        # precision per k in range
        tmp = sorted_df
        a = range(1, tmp.shape[0] + 1)
        tmp['position'] = pd.Series(a, index=tmp.index)
        print(tmp)
        large_k_p = tmp[(tmp['used_in_fix'] == 1.0)]['position'].tolist()
#         print('large_k_p :',large_k_p)
        unique_results = sorted_df['result'].unique().tolist()
        unique_results.sort()
#         print('unique_results',unique_results)
        for fk in large_k_p:
            k = int(fk)
            k_largest = unique_results[-k:]

            largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
            real_fixes_at_k = (largest_at_k['used_in_fix'] == 1.0).sum()

            p = float(real_fixes_at_k) / float(k)
            precision_at_k.append(p)
#         print('precision_at_k :',precision_at_k)
        average_precision = pd.Series(precision_at_k).mean()
#         print('average_precision :',average_precision)
        
        # accuracy
        for k in k_range:
            k_largest = unique_results[-k:]

            largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
            real_fixes_at_k = largest_at_k['used_in_fix'][(largest_at_k['used_in_fix'] == 1.0)].count()
            if real_fixes_at_k >= 1:
                accuracy_at_k[k] += 1
        print('accuracy_at_k :',accuracy_at_k)
        
        # reciprocal rank
        indexes_of_fixes = np.flatnonzero(sorted_df['used_in_fix'] == 1.0)
        if indexes_of_fixes.size == 0:
            reciprocal_ranks.append(0.0)
        else:
            first_index = indexes_of_fixes[0]
            reciprocal_rank = 1.0 / (first_index + 1)
            reciprocal_ranks.append(reciprocal_rank)
        # bug number
        bug_report_number += 1
        print('reciprocal_ranks :',reciprocal_ranks)
        if bug_report_number==15:
            break

# task1_list=[]
# for line in res:
#     print(line)
#     task1=dict()
#     task1['id_']=line[0]
#     task1['res']=line[1]
#     task1_list.append(task1)
# with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task2/sort_2_6_train_0524_nolabel.json','w',encoding='utf-8') as f_out:
#     for task1 in task1_list:
#         out_line = json.dumps(task1, ensure_ascii=False) + '\n'
#         f_out.write(out_line)

swt_all_results_df_before.reset_index(level=1, drop=True, inplace=True)

swt_all_results_df_before.head()





# swt_all_results_df_before

# calculate_metrics(swt_all_results_df_before)

# 使用tokenizedSource summary+description，按照CodeSearch组织数据

from nltk.corpus import stopwords
from collections import Counter
from nltk import wordpunct_tokenize
from nltk.stem.porter import PorterStemmer
from re import finditer
stemmer = PorterStemmer()

def camel_case_split(identifier):
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]
def mergeTokenziedSource(tokenizedSource):#dict 类型
    return ' '.join(tokenizedSource.keys())
removed = u'!"#%&\'()*+,-./:;<=>?@[\]^_`{|}~1234567890'
utf_translate_table = dict((ord(char), u' ') for char in removed)
stop_words = set(stopwords.words('english'))

def tokenize(text, stemmer):
    sanitized_text = text.translate(utf_translate_table)
    tokens = wordpunct_tokenize(sanitized_text)
    all_tokens = []
    for token in tokens:
        additional_tokens = camel_case_split(token)
        if len(additional_tokens)>1:
            for additional_token in additional_tokens:
                all_tokens.append(additional_token)
        all_tokens.append(token)
    return Counter([stemmer.stem(token) for token in all_tokens if token.lower() not in stop_words])

mergeTokenziedSource(tokenize('DirectoryDialog There is no disk in the drive . Please insert a disk into drive Device Harddisk0 DR0 Any operation which brings up a file dialog box rooted at "My Computer" tosses up a dialog box with the above error message whenever a harddrive is expanded . Click cancel or try again on dialog ( has cancel , try again , and continue ) twice , and then it opens up the disk , and proceeds normally . If you un - expand a drive , it does the same thing . Then again when you re - expand a drive . Examples : Import - > File System - > click Browse Feature . xml - > Export - > click Browse I have seen this all the way back through M4 . May have existed sooner . Currently using RC3 . Platform : Windows 2K SP3',stemmer))

def mergeTokenziedSource(tokenizedSource):#dict 类型
    return ' '.join(tokenizedSource.keys())
     
def generate(train_k_path,out_path):
    #生成fine-tune需要的数据集
    #train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv' 
    #out_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.txt'
    train_k=pd.read_csv(train_k_path)
    train_k.set_index(['bid','fid'], inplace = True)
    train_k.head(10)
    #取bid所对应的report
    bid_list=list(train_k.index.get_level_values(0).unique())
    len(bid_list)
    summarys=[]
    descriptions=[]
    for bug_report_id in bid_list:
        current_bug_report = bug_reports[bug_report_id]['bug_report']
        summarys.append(current_bug_report['summary'])
        descriptions.append(current_bug_report['description'])
    report_dataFrame=pd.DataFrame({'summary':summarys,'description':descriptions},index=bid_list)
    report_dataFrame.index.names=['bid']
    def remove_twoHeadWord(string):
        contents=string.split(' ')[2:]
        return ' '.join(contents)
    report_dataFrame['summary']=report_dataFrame['summary'].apply(remove_twoHeadWord)
    report_dataFrame.head()
    all_ast_index=list(train_k.index.get_level_values(1).unique())
    type(all_ast_index)
    all_ast_file_methodContent=[]
    for ast_index in all_ast_index:
    #     print(ast_index)
        ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
        all_ast_file_methodContent.append(mergeTokenziedSource(ast_file['tokenizedSource']))
    # all_ast_index
    all_ast_index_dataframe=pd.DataFrame({'tokenizedSource':all_ast_file_methodContent},index=all_ast_index)
    # print(all_ast_index_dataframe.head())
    print(all_ast_index_dataframe.shape)
    # all_ast_index_dataframe.set_index(["ast_index"], inplace=True)
    # all_ast_index_dataframe
    all_ast_index_dataframe.index.names=['fid']
    all_ast_index_dataframe.head(2)
    #开始连接字段
    all_methodContent_index_dataframe=all_ast_index_dataframe.join(train_k,how='inner')
    all_methodContent_index_dataframe.head()
    all_report_methodContent_index_dataframe=report_dataFrame.join(all_methodContent_index_dataframe,how='inner')
    all_report_methodContent_index_dataframe.head()
    
    task1_list=[]
    all_report_methodContent_index_dataframe.fillna("",inplace=True)
    def clean_string(string):
        outstring_list=re.findall(r'[\w"]+|[.,!?;{}:()\+\-\*\/=><"]',string)
        return ' '.join(outstring_list)
    sum_label=0
    for i,(index, row) in enumerate(all_report_methodContent_index_dataframe.iterrows()):
    #     print(index[0],index[1],row['summary'],row['description'],row['methodContent'],row['used_in_fix'])
        if len(row['tokenizedSource'])==0:
            continue
        guid='_'.join([index[0],index[1]])
        
#         q_paras=clean_string(row['summary']+' '+row['description'])
        q_paras=mergeTokenziedSource(tokenize(row['summary']+' '+row['description'],stemmer))
        c_paras=row['tokenizedSource']
#         print(i,'summary :',type(row['summary']),'description :',type(row['description']),'len summary :',len(q_paras.split()),'source :',len(c_paras.split()))
        summary_len.append(len(q_paras.split()))
        source_len.append(len(c_paras.split()))
        label=str(int(row['used_in_fix']))
        list_all=[label,guid,'hdj',q_paras,c_paras]
        task1_list.append('<CODESPLIT>'.join(list_all))
#         if i==10:
#             break
    print(task1_list[0])
    with open(out_path,'w',encoding='utf-8') as f_out:
        for task1 in task1_list:
            f_out.write(task1+'\n')
train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv' 
out_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.txt'    
summary_len=[]
source_len=[]
generate(train_k_path,out_path)



statis(summary_len)
statis(source_len)

# summary_len

# 按照CodeSearch组织数据结束

# 生成fine-tune阶段的数据 为每一个fold都生成训练数据

def generate_fine_tune(train_k_path,out_path):
    #生成fine-tune需要的数据集
    #train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv' 
    #out_path='/home/hdj/BERT-PLI-IJCAI2020/examples/task2/data_sample_1.json'
    train_k=pd.read_csv(train_k_path)
    train_k.set_index(['bid','fid'], inplace = True)
    train_k.head(10)
    #取bid所对应的report
    bid_list=list(train_k.index.get_level_values(0).unique())
    len(bid_list)
    summarys=[]
    descriptions=[]
    for bug_report_id in bid_list:
        current_bug_report = bug_reports[bug_report_id]['bug_report']
        summarys.append(current_bug_report['summary'])
        descriptions.append(current_bug_report['description'])
    report_dataFrame=pd.DataFrame({'summary':summarys,'description':descriptions},index=bid_list)
    report_dataFrame.index.names=['bid']
    def remove_twoHeadWord(string):
        contents=string.split(' ')[2:]
        return ' '.join(contents)
    report_dataFrame['summary']=report_dataFrame['summary'].apply(remove_twoHeadWord)
    report_dataFrame.head()
    all_ast_index=list(train_k.index.get_level_values(1).unique())
    type(all_ast_index)
    all_ast_file_methodContent=[]
    for ast_index in all_ast_index:
    #     print(ast_index)
        ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
        all_ast_file_methodContent.append(ast_file['methodContent'])
    # all_ast_index
    all_ast_index_dataframe=pd.DataFrame({'methodContent':all_ast_file_methodContent},index=all_ast_index)
    # print(all_ast_index_dataframe.head())
    print(all_ast_index_dataframe.shape)
    # all_ast_index_dataframe.set_index(["ast_index"], inplace=True)
    # all_ast_index_dataframe
    all_ast_index_dataframe.index.names=['fid']
    all_ast_index_dataframe.head(2)
    #开始连接字段
    all_methodContent_index_dataframe=all_ast_index_dataframe.join(train_k,how='inner')
    all_methodContent_index_dataframe.head()
    all_report_methodContent_index_dataframe=report_dataFrame.join(all_methodContent_index_dataframe,how='inner')
    all_report_methodContent_index_dataframe.head()
    
    task1_list=[]
    all_report_methodContent_index_dataframe.fillna("",inplace=True)
    def clean_string(string):
        outstring_list=re.findall(r'[\w"]+|[.,!?;{}:()\+\-\*\/=><"]',string)
        return ' '.join(outstring_list)
    sum_label=0
    for i,(index, row) in enumerate(all_report_methodContent_index_dataframe.iterrows()):
    #     print(index[0],index[1],row['summary'],row['description'],row['methodContent'],row['used_in_fix'])
        if len(row['methodContent'])==0:
            continue
        guid='_'.join([index[0],index[1]])
#         print(i,'summary :',type(row['summary']),'description :',type(row['description']))
#         q_paras=[clean_string(row['summary']),clean_string(row['description'])]
#         c_paras=[clean_string(string) for string in row['methodContent']]
        q_paras=[mergeTokenziedSource(tokenize(row['summary'],stemmer)),mergeTokenziedSource(tokenize(row['description'],stemmer))]
        c_paras=[mergeTokenziedSource(tokenize(string,stemmer)) for string in row['methodContent']]
        
        label=int(row['used_in_fix'])

        for q in q_paras:
            if q=="":
                continue
            for c in c_paras:
                if c=="":
                    continue
                task1=dict()

                task1['guid']=guid
                task1['text_a']=q
                task1['text_b']=c
                task1['label']=label
                sum_label+=label
                task1_list.append(task1)
    #     print(task1)
    #     if i==2:
    #         break
    print(len(task1_list),sum_label,sum_label/len(task1_list))
    task1_list[0]
    label=0
    label_0=0
    for task in task1_list:
    #     print(task)
        label+=task['label']
        if task['label']==0:
            label_0+=1
    #     break
    print(label)
    with open(out_path,'w',encoding='utf-8') as f_out:
        for task1 in task1_list:
            out_line = json.dumps(task1, ensure_ascii=False) + '\n'
            f_out.write(out_line)
train_k_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv'
out_path='/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1_fine_tune.txt'
generate_fine_tune(train_k_path,out_path)

train_k=pd.read_csv('/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1.csv')
train_k.set_index(['bid','fid'], inplace = True)
train_k.head(10)
#取bid所对应的report
bid_list=list(train_k.index.get_level_values(0).unique())

len(bid_list)
summarys=[]
descriptions=[]
for bug_report_id in bid_list:
    current_bug_report = bug_reports[bug_report_id]['bug_report']
    summarys.append(current_bug_report['summary'])
    descriptions.append(current_bug_report['description'])
report_dataFrame=pd.DataFrame({'summary':summarys,'description':descriptions},index=bid_list)
report_dataFrame.index.names=['bid']
def remove_twoHeadWord(string):
    contents=string.split(' ')[2:]
    return ' '.join(contents)
report_dataFrame['summary']=report_dataFrame['summary'].apply(remove_twoHeadWord)
report_dataFrame.head()

ast_file = pickle.loads(swt_ast_cache_collection_db['ca0faf5d670090c4b5c3f4dccbcedefe28fb6b01'])
ast_file

all_ast_index=list(train_k.index.get_level_values(1).unique())
type(all_ast_index)
all_ast_file_methodContent=[]
count=0
sourceLen=[]
for ast_index in all_ast_index:
#     print(ast_index)
    ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
    print(ast_index,len(ast_file['tokenizedMethods']),len(ast_file['tokenizedSource']))#TODO
    if len(ast_file['tokenizedMethods'])==0:
        count+=1
    sourceLen.append(len(ast_file['tokenizedSource']))
    all_ast_file_methodContent.append(ast_file['methodContent'])
print(count)

576/len(all_ast_index)


# all_ast_index
all_ast_index_dataframe=pd.DataFrame({'methodContent':all_ast_file_methodContent},index=all_ast_index)
# print(all_ast_index_dataframe.head())
print(all_ast_index_dataframe.shape)
# all_ast_index_dataframe.set_index(["ast_index"], inplace=True)
# all_ast_index_dataframe
all_ast_index_dataframe.index.names=['fid']
all_ast_index_dataframe.head(2)

all_ast_index_dataframe['methodContent']

#开始连接字段
all_methodContent_index_dataframe=all_ast_index_dataframe.join(train_k,how='inner')
all_methodContent_index_dataframe.head()

all_report_methodContent_index_dataframe=report_dataFrame.join(all_methodContent_index_dataframe,how='inner')
all_report_methodContent_index_dataframe.head()

sum(all_report_methodContent_index_dataframe['used_in_fix'])/all_report_methodContent_index_dataframe.shape[0]

all_report_methodContent_index_dataframe.fillna("",inplace=True)

task1_list=[]
all_report_methodContent_index_dataframe.fillna("",inplace=True)
def clean_string(string):
    outstring_list=re.findall(r'[\w"]+|[.,!?;{}:()\+\-\*\/=><"]',string)
    return ' '.join(outstring_list)
sum_label=0
for i,(index, row) in enumerate(all_report_methodContent_index_dataframe.iterrows()):
#     print(index[0],index[1],row['summary'],row['description'],row['methodContent'],row['used_in_fix'])
    if len(row['methodContent'])==0:
        continue
    guid='_'.join([index[0],index[1]])
    print(i,'summary :',type(row['summary']),'description :',type(row['description']))
    q_paras=[clean_string(row['summary']),clean_string(row['description'])]
    c_paras=[clean_string(string) for string in row['methodContent']]
    label=int(row['used_in_fix'])
    
    for q in q_paras:
        if q=="":
            continue
        for c in c_paras:
            if c=="":
                continue
            task1=dict()
            
            task1['guid']=guid
            task1['text_a']=q
            task1['text_b']=c
            task1['label']=label
            sum_label+=label
            task1_list.append(task1)
#     print(task1)
#     if i==2:
#         break


print(len(task1_list),sum_label,sum_label/len(task1_list))
task1_list[0]
label=0
label_0=0
for task in task1_list:
#     print(task)
    label+=task['label']
    if task['label']==0:
        label_0+=1
#     break
label
with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task2/data_sample.json','w',encoding='utf-8') as f_out:
    for task1 in task1_list:
        out_line = json.dumps(task1, ensure_ascii=False) + '\n'
        f_out.write(out_line)

label_0+190582

# 生成fine-tune阶段的数据结束

bid_list=list(all_results_df_top_k.index.get_level_values(0).unique())
len(bid_list)
bid_list[0], bug_reports[bid_list[0]]['bug_report']['summary'], bug_reports[bid_list[0]]['bug_report']['description']

bug_reports
summarys=[]
descriptions=[]
for bug_report_id in bid_list:
    current_bug_report = bug_reports[bug_report_id]['bug_report']
    summarys.append(current_bug_report['summary'])
    descriptions.append(current_bug_report['description'])
report_dataFrame=pd.DataFrame({'summary':summarys,'description':descriptions},index=bid_list)
report_dataFrame.index.names=['bid']
def remove_twoHeadWord(string):
    contents=string.split(' ')[2:]
    return ' '.join(contents)
report_dataFrame['summary']=report_dataFrame['summary'].apply(remove_twoHeadWord)
report_dataFrame.head()

# all_results_df_top_k.index

bug_reports['9cdad39']['bug_report']

# tomcat_ast_file = pickle.loads(tomcat_ast_cache_collection_db['b6437f63a95972f4a2d3de68d4c9d5691c5792fd'])
# tomcat_ast_file

len(all_ast_index)

all_ast_index=list(all_results_df_top_k.index.get_level_values(1).unique())
type(all_ast_index)
all_ast_file_methodContent=[]
for ast_index in all_ast_index:
#     print(ast_index)
    ast_file = pickle.loads(swt_ast_cache_collection_db[ast_index])
    all_ast_file_methodContent.append(ast_file['methodContent'])

count=0
for method in all_ast_file_methodContent:
    if len(method)==0:
        count+=1
print(count,len(all_ast_file_methodContent))

# all_ast_index
all_ast_index_dataframe=pd.DataFrame({'methodContent':all_ast_file_methodContent},index=all_ast_index)
# print(all_ast_index_dataframe.head())
print(all_ast_index_dataframe.shape)
# all_ast_index_dataframe.set_index(["ast_index"], inplace=True)
# all_ast_index_dataframe
all_ast_index_dataframe.index.names=['fid']
all_ast_index_dataframe.head(2)

# all_ast_index_dataframe['methodContent'].loc['6f547898bd6d011af1e621a2c002f10c39856cd4']

# all_ast_index_dataframe['methodContent'].loc['b5f5e1a2070ba15a8d5a8cd955de857225e0815b']

all_ast_index_dataframe.columns

# all_ast_index_dataframe.to_csv('')
# tomcat_testing_fold_1.lef
# pd.merge(tomcat_testing_fold_1,all_ast_index_dataframe,right_on='ast_index',left_index=True, how='outer')
all_methodContent_index_dataframe=all_ast_index_dataframe.join(all_results_df_top_k,how='inner')

all_methodContent_index_dataframe.head()

report_dataFrame.head()



sum(all_report_methodContent_index_dataframe.loc['1839d9b']['used_in_fix'])

all_report_methodContent_index_dataframe=report_dataFrame.join(all_methodContent_index_dataframe,how='inner')
all_report_methodContent_index_dataframe.head()

all_report_methodContent_index_dataframe=all_report_methodContent_index_dataframe[['summary','description','methodContent','used_in_fix']]
all_report_methodContent_index_dataframe.head()

all_report_methodContent_index_dataframe.fillna("",inplace=True)

task1_list=[]
def clean_string(string):
    outstring_list=re.findall(r'[\w"]+|[.,!?;{}:()\+\-\*\/=><"]',string)
    return ' '.join(outstring_list)
for i,(index, row) in enumerate(all_report_methodContent_index_dataframe.iterrows()):
#     print(index[0],index[1],row['summary'],row['description'],row['methodContent'],row['used_in_fix'])
    task1=dict()
    task1['guid']='_'.join([index[0],index[1]])
    task1['q_paras']=[clean_string(row['summary']),clean_string(row['description'])]
    task1['c_paras']=[clean_string(string) for string in row['methodContent']]
    task1['label']=int(row['used_in_fix'])
    task1_list.append(task1)
#     if i==100:
#         break
with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/case_para_sample.json','w',encoding='utf-8') as f_out:
    for task1 in task1_list:
        out_line = json.dumps(task1, ensure_ascii=False) + '\n'
        f_out.write(out_line)

guids = []
input_ids = []
attention_mask = []
token_type_ids = []
for i in range(3):
    input_ids_item = []
    attention_mask_item = []
    token_type_ids_item = []
    for j in range(2):
        input_ids_row = []
        attention_mask_row = []
        token_type_ids_row = []
        for k in range(10):
            input_ids_row.append([1,1,1])
            attention_mask_row.append([2,2,2])
            token_type_ids_row.append([3,3,3])
        input_ids_item.append(input_ids_row)
        attention_mask_item.append(attention_mask_row)
        token_type_ids_item.append(token_type_ids_row)
    guids.append('guid')
    input_ids.append(input_ids_item)
    attention_mask.append(attention_mask_item)
    token_type_ids.append(token_type_ids_item)

import torch
print(guids)
print(input_ids)
print(attention_mask)
input_ids = torch.LongTensor(input_ids)
attention_mask = torch.LongTensor(attention_mask)
token_type_ids = torch.LongTensor(token_type_ids)

input_ids.shape

bert = RobertaModel.from_pretrained("microsoft/codebert-base")

for k in range(input_ids.shape[0]):
    for i in range(0,2,2):
        _,lst=bert(input_ids[k,i:i+2].view(-1,10),attention_mask=attention_mask[k,i:i+2].view(-1,10))
        

_.shape

lst.shape

# 10.10开始跑的

25000/1000,25*10/60

for k in range(input_ids.size()[0]):
    for i in range(0,2,2):
        input_id_tmp=input_ids[k,i:i+2]
        print(k,input_id_tmp.shape)
        input_id_view=input_id_tmp.view(-1,3)
        print(k,input_id_view.shape)

with open('/data/hdj/tracking_buggy_files/joblib_memmap_swt/train_1_tokenzied2_6_stage2.json','r',encoding='utf-8') as f_in:
    label=0
    i=0
    for line in f_in:
        line=eval(line)
        label+=line['label']
        i+=1
    print(label,i,label/i)

# 组装成rnn需要的数据格式


label_1=0
j=0
for key,val in guid_label.items():
    j+=1
    label_1+=val
print(label_1)
print(j)
print(label_1/j)

len(guid_label)
# guid_label

#这里是在oversample
with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/embedding_sample.json','r',encoding='utf-8') as f_in:
    with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/embedding_sample_label.json','w',encoding='utf-8') as f_out:
        for line in f_in:
            line_dict=eval(line)
    #         for key,val in line.items():
            if line_dict['label']==1:
                for i in range(10):
                    f_out.write(line)
            else:
                f_out.write(line)


#把500个文件分割成20个，看看哪里效果很差 为啥
import collections
out_path_label='/home/hdj/BERT-PLI-IJCAI2020/result/sort_floss_2_15_train_0525_2hard_1easy_right7model_label.json'
bid_list=[]
used_in_fix=[]
result=[]
all_data=collections.defaultdict(list)
with open(out_path_label,'r',encoding='utf-8') as f_in:
    for i,line in enumerate(f_in):
#         line=eval(line)
        all_data[i//1000].append(line)

import pandas as pd
import numpy as np
calculate_metrics(all_data[0])

def calculate_metrics(f_in, k_range=range(1, 21)):
    for line in f_in:
        line=eval(line)
        bid_list.append(line['id_'].split('_')[0])
        used_in_fix.append(line['label'])
        result.append(line['res'][1])
    stage2_result=pd.DataFrame()
    stage2_result['bid']=bid_list
    stage2_result['used_in_fix']=used_in_fix
    stage2_result['result']=result
    stage2_result.set_index('bid',inplace=True)
    stage2_result.shape
    verification_df=stage2_result
    average_precision_per_bug_report = []
    reciprocal_ranks = []
    # calculate per each query (bug report)
    accuracy_at_k = dict.fromkeys(k_range, 0)
    bug_report_number = 0
    for bug_report, bug_report_files_dataframe in verification_df.groupby(level=0, sort=False):
        print(bug_report)
        min_fix_result = bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix'] == 1.0]['result'].min()
        bug_report_files_dataframe2 = bug_report_files_dataframe[bug_report_files_dataframe["result"] >= min_fix_result]
        sorted_df = bug_report_files_dataframe2.sort_values(ascending=False, by=['result'])
        if sorted_df.shape[0] == 0:
            sorted_df = bug_report_files_dataframe.copy().sort_values(ascending=False, by=['result'])
            # print((bug_report_files_dataframe['used_in_fix'] == 1.0).sum())

        precision_at_k = []
        # precision per k in range
        tmp = sorted_df
        a = range(1, tmp.shape[0] + 1)
        tmp['position'] = pd.Series(a, index=tmp.index)
        
        large_k_p = tmp[(tmp['used_in_fix'] == 1.0)]['position'].tolist()
        unique_results = sorted_df['result'].unique().tolist()
        unique_results.sort()
        for fk in large_k_p:
            k = int(fk)
            k_largest = unique_results[-k:]

            largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
            real_fixes_at_k = (largest_at_k['used_in_fix'] == 1.0).sum()

            p = float(real_fixes_at_k) / float(k)
            precision_at_k.append(p)

        # average precision is sum of k precisions divided by K
        # K is set of positions of relevant documents in the ranked list
        average_precision = pd.Series(precision_at_k).mean()
        # average_precision = pd.Series(precision_at_k).sum() / float(large_k)
        average_precision_per_bug_report.append(average_precision)

        # accuracy
        for k in k_range:
            k_largest = unique_results[-k:]

            largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
            real_fixes_at_k = largest_at_k['used_in_fix'][(largest_at_k['used_in_fix'] == 1.0)].count()
            if real_fixes_at_k >= 1:
                accuracy_at_k[k] += 1

        # reciprocal rank
        indexes_of_fixes = np.flatnonzero(sorted_df['used_in_fix'] == 1.0)
        if indexes_of_fixes.size == 0:
            reciprocal_ranks.append(0.0)
        else:
            first_index = indexes_of_fixes[0]
            reciprocal_rank = 1.0 / (first_index + 1)
            reciprocal_ranks.append(reciprocal_rank)
        # bug number
        bug_report_number += 1

        del bug_report, bug_report_files_dataframe

    # print("average_precision_per_bug_report", average_precision_per_bug_report)
    mean_average_precision = pd.Series(average_precision_per_bug_report).mean()
    # print('mean average precision', mean_average_precision)
    mean_reciprocal_rank = pd.Series(reciprocal_ranks).mean()
    # print('mean reciprocal rank', mean_reciprocal_rank)
    print('accuracy_at_k :',accuracy_at_k)
    for k in k_range:
        accuracy_at_k[k] = accuracy_at_k[k] / bug_report_number
        # print('accuracy for k', accuracy_at_k[k], k)
    return accuracy_at_k, mean_average_precision, mean_reciprocal_rank

# swt_testing_fold_1.rename(index={0:'bid',1:'fid'},  inplace=True)
# swt_testing_fold_1.index.names = ['bid','fid']
#TODO 增加label
import json
import numpy as np
import pandas as pd
guid_label=dict()
# with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/case_para_sample_8.json','r',encoding='utf-8') as f_in:
with open('/data/hdj/BERT_PLI/test_2_tokenzied2_6_stage2.json','r',encoding='utf-8') as f_in:
    label=0
    i=0
    for line in f_in:
        line=eval(line)
        i+=1
        guid_label[line['guid']]=line['label']

pred_no_label='/home/hdj/BERT-PLI-IJCAI2020/result/sort_floss_2_15_train_0525_2hard_1easy_right7model_less.json'
out_path_label='/home/hdj/BERT-PLI-IJCAI2020/result/sort_floss_2_15_train_0525_2hard_1easy_right7model_less_label.json'
with open(pred_no_label,'r',encoding='utf-8') as f_in:
    with open(out_path_label,'w',encoding='utf-8') as f_out:
        for line in f_in:
            line=eval(line)
            line['label']=guid_label[line['id_']]
            out_line = json.dumps(line, ensure_ascii=False) + '\n'
            f_out.write(out_line)
print(i)
bid_list=[]
used_in_fix=[]
result=[]
with open(out_path_label,'r',encoding='utf-8') as f_in:
    for line in f_in:
        line=eval(line)
        bid_list.append(line['id_'].split('_')[0])
        used_in_fix.append(line['label'])
        result.append(line['res'][1])
stage2_result=pd.DataFrame()
stage2_result['bid']=bid_list
stage2_result['used_in_fix']=used_in_fix
stage2_result['result']=result
stage2_result.set_index('bid',inplace=True)
stage2_result.shape
def calculate_metrics(f_in, k_range=range(1, 21)):
    for line in f_in:
        line=eval(line)
        bid_list.append(line['id_'].split('_')[0])
        used_in_fix.append(line['label'])
        result.append(line['res'][1])
    stage2_result=pd.DataFrame()
    stage2_result['bid']=bid_list
    stage2_result['used_in_fix']=used_in_fix
    stage2_result['result']=result
    stage2_result.set_index('bid',inplace=True)
    stage2_result.shape
    verification_df=stage2_result
    average_precision_per_bug_report = []
    reciprocal_ranks = []
    # calculate per each query (bug report)
    accuracy_at_k = dict.fromkeys(k_range, 0)
    bug_report_number = 0
    for bug_report, bug_report_files_dataframe in verification_df.groupby(level=0, sort=False):
        print(bug_report)
        min_fix_result = bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix'] == 1.0]['result'].min()
        bug_report_files_dataframe2 = bug_report_files_dataframe[bug_report_files_dataframe["result"] >= min_fix_result]
        sorted_df = bug_report_files_dataframe2.sort_values(ascending=False, by=['result'])
        if sorted_df.shape[0] == 0:
            sorted_df = bug_report_files_dataframe.copy().sort_values(ascending=False, by=['result'])
            # print((bug_report_files_dataframe['used_in_fix'] == 1.0).sum())

        precision_at_k = []
        # precision per k in range
        tmp = sorted_df
        a = range(1, tmp.shape[0] + 1)
        tmp['position'] = pd.Series(a, index=tmp.index)
        
        large_k_p = tmp[(tmp['used_in_fix'] == 1.0)]['position'].tolist()
        unique_results = sorted_df['result'].unique().tolist()
        unique_results.sort()
        for fk in large_k_p:
            k = int(fk)
            k_largest = unique_results[-k:]

            largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
            real_fixes_at_k = (largest_at_k['used_in_fix'] == 1.0).sum()

            p = float(real_fixes_at_k) / float(k)
            precision_at_k.append(p)

        # average precision is sum of k precisions divided by K
        # K is set of positions of relevant documents in the ranked list
        average_precision = pd.Series(precision_at_k).mean()
        # average_precision = pd.Series(precision_at_k).sum() / float(large_k)
        average_precision_per_bug_report.append(average_precision)

        # accuracy
        for k in k_range:
            k_largest = unique_results[-k:]

            largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
            real_fixes_at_k = largest_at_k['used_in_fix'][(largest_at_k['used_in_fix'] == 1.0)].count()
            if real_fixes_at_k >= 1:
                accuracy_at_k[k] += 1

        # reciprocal rank
        indexes_of_fixes = np.flatnonzero(sorted_df['used_in_fix'] == 1.0)
        if indexes_of_fixes.size == 0:
            reciprocal_ranks.append(0.0)
        else:
            first_index = indexes_of_fixes[0]
            reciprocal_rank = 1.0 / (first_index + 1)
            reciprocal_ranks.append(reciprocal_rank)
        # bug number
        bug_report_number += 1

        del bug_report, bug_report_files_dataframe

    # print("average_precision_per_bug_report", average_precision_per_bug_report)
    mean_average_precision = pd.Series(average_precision_per_bug_report).mean()
    # print('mean average precision', mean_average_precision)
    mean_reciprocal_rank = pd.Series(reciprocal_ranks).mean()
    # print('mean reciprocal rank', mean_reciprocal_rank)
    print('accuracy_at_k :',accuracy_at_k)
    for k in k_range:
        accuracy_at_k[k] = accuracy_at_k[k] / bug_report_number
        # print('accuracy for k', accuracy_at_k[k], k)
    return accuracy_at_k, mean_average_precision, mean_reciprocal_rank

calculate_metrics(stage2_result)

25000/3

# 组装成rnn需要的数据格式 完成

# output_data=dict()
# import json
# with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/embedding_sample_0524_test.json','r',encoding='utf-8') as f_in:
#     with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/embedding_sample_0524_test_label.json','w',encoding='utf-8') as f_out:
#         for line in f_in:
#             line=eval(line)
#             output_data['guid']=line['id_']
#             output_data['res']=line['res']
#             output_data['label']=guid_label[line['id_']]
# #             for key,val in output_data.items():
# #                 print(key)
# #             print(type(line))
#             out_line = json.dumps(output_data, ensure_ascii=False) + '\n'
#             if guid_label==1:
#                 for i in range(10):
#                     f_out.write(out_line)
#             else:
#                 f_out.write(out_line)
# #             break

c_paras_len=[]
c_paras_single_len=[]
for task1 in task1_list:
    c_paras=task1['c_paras']
    c_paras_len.append(len(c_paras))
    single_len=[]
    for para in c_paras:
        single_len.append(len(para.split(' ')))
    c_paras_single_len.append(single_len)


# task1_list[8]

# for one in c_paras_single_len:
#     print(len(one))

a = np.array(c_paras_len)
print('code 的 10%', np.percentile(a, 10))
print('20 :', np.percentile(a, 20))
print('30 :', np.percentile(a, 30))
print('40 :', np.percentile(a, 40))
print('50 :', np.median(a))
print('60 :', np.percentile(a, 60))
print('70 :', np.percentile(a, 70))
print('80 :', np.percentile(a, 80))
print('90 :', np.percentile(a, 90))

25000/(60*60)

val_count=data['len'].value_counts()

val_count.head()

task1_list[25000-50]['guid'],task1_list[25000-2]['guid'],task1_list[25000-1]['guid'],task1_list[25000]['guid']

with open('/home/hdj/BERT-PLI-IJCAI2020/examples/task1/case_para_sample.json','w',encoding='utf-8') as f_out:
    for task1 in task1_list:
        out_line = json.dumps(task1, ensure_ascii=False) + '\n'
        f_out.write(out_line)

# public boolean isSubtype ( ObjectType type , ObjectType possibleSupertype ) throws ClassNotFoundException { if ( DEBUG_QUERIES ) { System . out . println ( "isSubtype: check " + type + " subtype of " + possibleSupertype ) ; } if ( type . equals ( possibleSupertype ) ) { if ( DEBUG_QUERIES ) { System . out . println ( "  ==> yes, types are same" ) ; } return true ; } ClassDescriptor typeClassDescriptor = DescriptorFactory . getClassDescriptor ( type ) ; ClassDescriptor possibleSuperclassClassDescriptor = DescriptorFactory . getClassDescriptor ( possibleSupertype ) ; return isSubtype ( typeClassDescriptor , possibleSuperclassClassDescriptor ) ; }
# public boolean isSubtype(ObjectType type, ObjectType possibleSupertype) throws ClassNotFoundException {
#         if (DEBUG_QUERIES) {
#             System.out.println("isSubtype: check " + type + " subtype of " + possibleSupertype);
#         }

#         if (type.equals(possibleSupertype)) {
#             if (DEBUG_QUERIES) {
#                 System.out.println("  ==> yes, types are same");
#             }
#             return true;
#         }
#         ClassDescriptor typeClassDescriptor = DescriptorFactory.getClassDescriptor(type);
#         ClassDescriptor possibleSuperclassClassDescriptor = DescriptorFactory.getClassDescriptor(possibleSupertype);

#         return isSubtype(typeClassDescriptor, possibleSuperclassClassDescriptor);
#     }

# {"guid": "005_001", "q_paras": ["Strickland, J. : This is an application for judicial review of two decisions of the Canadian Human Rights Commission (the Commission), both dated August 27, 2014, declining to deal with the Applicant's complaints, pursuant to s 41(1)(a) of the Canadian Human Rights Act , RSC 1985, c H-6 ( CHR Act ), because the Applicant had not exhausted grievance or review procedures otherwise reasonably available to him. The application for judicial review is brought pursuant to s 18.1 of the Federal Courts Act , RSC 1985, c F-7. Background", "The Applicant is a member of the Canadian Forces (CF), the Respondent herein as represented by the Attorney General of Canada. He alleges that in 2010, while he was serving in Haiti, his immediate superior officer discriminated against him, largely on racial grounds. In June 2010, the Applicant filed a complaint with the CF Ombudsman who decided not to deal with the case because other avenues of redress had not yet been exhausted.", "In August 2010 the Applicant met with his Commanding Officer (CO) and the harassment complaint was discussed. His CO subsequently wrote to him on two occasions in September 2010, requesting additional information so as to determine whether the Applicant's allegations constituted a harassment complaint pursuant to the Harassment Prevention and Resolution Guidelines . No response was received from the Applicant.", "By letter of October 5, 2010 counsel for the Applicant advised the CO, amongst other things, that the Applicant did not intend to avail himself of a harassment complaint pursuant to the Department of Nation Defence/CF Harassment Prevention and Resolution policy but reserved his right to make any such complaints in another forum.", "In December 2010 the Applicant filed a complaint with the Commission. By decisions dated July 6, 2011 the Commission decided not to deal with the complaint at that time, pursuant to s 41(1)(a) of the CHR Act , as the Applicant ought first to exhaust grievance or review procedures otherwise reasonably available to him. It stated that at the end of the grievance or review process, the Applicant could ask the Commission to reactivate the complaint. The Commission's decisions of July 6, 2011 are not under review.", "The Applicant then engaged in the CF Redress of Grievance process. On November 29, 2011 the Commander of Canadian Forces Joint Support Group released terms of reference, thereby convening an administrative investigation into the Applicant's allegations of racist conduct, discrimination and harassment. The Administrative Investigation report of the Initial Authority (IA) was released on April 20, 2012. The Applicant was dissatisfied with the IA decision and requested that his grievance be forwarded for determination by the Final Authority (FA).", "On July 4, 2012 the Applicant's grievance was referred to the Canadian Forces Grievance Board (CFGB) which released its findings and recommendations on September 14, 2012. By letter of March 4, 2013, the Chief of the Defence Staff advised that he concurred with the CFGB findings and recommendations, which included that there were important evidentiary omissions and procedural errors that compromised the integrity of the process and resulted in a fatally flawed Administrative Investigation and, therefore, an equally flawed IA decision. The Chief of the Defence Staff also stated that he was concerned that the Applicant had chosen not to avail himself of the CF harassment process and that this had contributed to the CFGB's findings. Given his finding that the IA decision was invalid, the Chief of the Defence Staff concluded that he could not make a properly informed decision on the validity of the allegations and, therefore, that it was imperative that the Applicant initiate the harassment investigation process. The Applicant was given 90 days to do so. The Chief of the Defence Staff stated that when the investigation was complete the Applicant could grieve its outcome if its conclusions and recommendations were not to his satisfaction.", "On May 14, 2013 the Applicant submitted a copy of his original grievance, not a harassment complaint. The Responsible Officer (RO) wrote to the Applicant on May 22, 2013, requesting submission of a proper complaint and inclusion of any information necessary to enable the conduct of a thorough and complete investigation of the allegations of harassment. The RO sent a second letter on July 19, 2013. The Applicant did not respond to either letter.", "On December 24, 2012, while the grievance process was in progress, the Applicant requested the Commission to reactivate his complaint. While the Commission was considering whether s 41(1)(a) applied to the complaint, the FA decision was issued. The Applicant filed an application for judicial review of the FA on April 24, 2012. A November 4, 2013 Section 40/41 Report recommended that the Commission not deal with the complaint while the judicial review was before this Court. On January 23, 2014, before the Commission decided that matter, the Applicant discontinued his application for judicial review of the FA and asked the Commission to deal with his complaint.", "On May 6, 2014 two Section 40/41 Reports were prepared. The first concerned the Applicant's complaint against the CF (file no 20101251) and the second concerned his complaint against his CO (file no 20101252) (collectively, the Section 40/41 Report). Substantially similar Records of Decision Under Section 40/41 were issued by the Commission on August 27, 2014, for both matters (collectively, the Decision). For the purposes of this judicial review, all references shall be to the Section 40/41 Report and decision of the Commission pertaining to the complaint against the CF.", "On September 26, 2014 the Applicant filed his application for judicial review of the Commission's Decision. Decision Under Review", "The Commission decided not to deal with the complaint pursuant to s 41(1)(a) of the CHR Act , as the Applicant ought to have exhausted grievance or review procedures otherwise reasonably available to him. A decision under s 41(1)(d) of the CHR Act was, therefore, unnecessary. The Commission adopted the conclusion of the May 6, 2014 Section 40/41 Report: The complainant's allegations are serious, and in the public interest. On the other hand, the harassment complaint process was reasonably available to the complainant after the Final Authority decision in 2013. The Final Authority decision included measures to be taken as part of the harassment complaint process in order to address the concerns with timeliness and fairness from the preceding administrative investigation. The complainant was provided multiple opportunities to participate in this harassment complaint process, but he chose not to. He was clearly aware of the inadequacies of the grievance form to initiate a harassment complaint as several attempts were made to obtain the correct information from him. Based on the above, the failure to exhaust that process was attributable to him.", "In rendering its decision the Commission reviewed the December 8, 2010 complaint form, the Section 40/41 Report of May 6, 2014 and Supplemental Section 40/41 Report of June 12, 2014, the Applicant's submissions of June 17, 2014, the CF's submissions of July 2, 2014, and cross-disclosure submissions of the CF dated July 25, 2014.", "The Section 40/41 Report set out the background to the complaint and summarized the positions of the parties based on their submissions. It noted that, based on the CF's position, it appeared that the remaining alternative redress process, the CF's harassment complaint process, was no longer available to the Applicant because he did not file a formal harassment complaint prior to the deadline set out in the FA. The question to be answered was whether the harassment complaint process was reasonably available to the Applicant and, if so, whether he should have exhausted that procedure. And, if that process ought to have been exhausted, whether the failure to do so was solely attributable to the Applicant in accordance with s 42(2) of the CHR Act .", "On the first point, it is expedient to reproduce particular paragraphs of the report as it sets out relevance background information as well as its analysis: Was the harassment complaint process reasonably available to the complainant? 64. Some relevant background information can be found in the \"Findings and Recommendations\" of the Canadian Forces Grievance Board (\"CFGB\"), dated September 14, 2012, which was provided by the complainant. In August 2011, after the Commission's decision not to deal with his complaint at that time, the complainant filed a grievance with the respondent. In March 2012, a final report on the Administrative investigation of the complainant's grievance was issued and disclosed to the grievor who provided comments. On April 20, 2012, the Initial Authority decision was rendered, finding \"professional shortcomings\" on the part of the superior officer but not granting the grievor's requests for apologies, damages, or legal fees. The complainant was unsatisfied with this decision and asked for a decision by the Final Authority. The matter was referred to the CFGB for review and recommendations. 65. The CFGB found that because a harassment investigation did not occur, the Initial Authority did not have adequate information upon which to make his decision, even though he attempted to insert elements of a harassment investigation into the grievance process. This investigation was found to have various defects, such as not all witnesses were interviewed, not all relevant information appeared to be considered, and the fact that the superior officer, alleged to be the harasser, was not provided an opportunity to respond to the investigation. For these reasons, the CFGB recommended that the Final Authority not make a final decision on the matter, but order a new harassment investigation be initiated. It also noted that the respondent's policies should be amended to ensure that harassment complaints are required to be dealt with under the harassment policy prior to resorting to the grievance process, given that neither the CFGB nor the CDS is equipped to conduct its own investigation in such situations. 66. On March 4, 2013, the Final Authority rendered his decision. Referring to the CFGB findings and recommendations, the CDS found that there were \" important evidentiary omissions and procedural errors that compromised the integrity of the process and resulted in what the CFGB quite rightly referred to as a 'fatally flawed' administrative investigation and consequently led to an equally flawed IA decision based solely on that investigation.\" The CDS determined that due to these errors and the incompleteness of the underlying investigation, he could not render \" ...a properly informed decision on the validity and reliability of the allegations... \" 67. The CDS also raised concerns that a harassment investigation did not occur. In the Final Authority decision, the CDS stated: Yet, I am troubled by your allegations and most particularly, by the lack of a proper investigation into their validity. As I mentioned, the CF takes this issue very seriously and consequently, it is imperative that you initiate the harassment investigation process by submitting a formal harassment complaint to the CO of 4 CFMCU. You have 90 days from the receipt of this letter to make your submission. The CO of 4 CFMCU shall, upon receipt of your harassment complaint, initiate a harassment investigation into the allegations therein. Let me be very clear on this point: an investigation will not commence until there is a formal harassment complaint submitted to the CO of 4 CFMCU and you commit, in writing, that you will fully participate in such an investigation to ensure that a thorough and impartial analysis may determine the legitimacy of your allegations. If you do not respond within 90 days, there will be no further action taken on your alleged complaints. Given the complexities of this case, compounded by the length of time that has elapsed since the alleged incidents, the CO of the 4 CFMCU might consider engaging a private consulting firm to assist with the investigation. Such corporations specialize in the investigation of harassment cases. Additionally, a monthly situation report will be sent to the Director General Canadian forces [ sic ] Grievance Authority (DGCFGA) Implementation Officer on the status of the forthcoming harassment investigation. When the investigation has been completed, you may grieve its outcome if the conclusions and recommendations are not to your satisfaction. 68. The CDS suggested that a private consulting firm be engaged to investigate the matter. The complainant is of the view that this demonstrates that the respondent is unable to investigate a complaint of this complexity. He suggests that this is further supported by the finding of the CFGB that the initial administrative investigation to be [ sic ] ' fatally flawed '. 69. The lack of procedural fairness in the IA decision was acknowledged in the Final Authority decision. The CDS attempted to address the concerns for procedural fairness and expediency with his recommendations: a) that the matter be dealt with through a formal harassment complaint; and, b) that they hire an outside firm to investigate the matter. 70. The respondent's \"Harassment Prevention and Resolution Guidelines\" (the Guidelines) outline a complaint process which is overseen by Responsible Officers (ROs) in the respondent's organization. The Guidelines define harassment the same as it is defined under the Act. The Guidelines further provide that other workplace harassment may also be addressed through this process. Once a formal harassment complaint is received, the RO oversees a situational assessment to determine whether or not the complaint is related to harassment - if it is, efforts are made to resolve the matter, identify other possible redress avenues such as mediation, and the appropriate action(s) will be initiated. If mediation is not appropriate, an administrative investigation is to be undertaken by a harassment investigator. The harassment investigator prepares a draft report on which both parties may comment before the final report is completed and submitted to the RO for decision. If any person is not in agreement with the RO's decision, they may file a grievance. Although the harassment and grievance processes do not provide for an independent third party decision-maker, matters dealing with harassment and racist conduct are referred to an independent body (the CFGB, now the MGERC) for findings and recommendation prior to the Final Authority decision being rendered. In addition, a review by the Federal Court is also available. 71. The complainant states that only the Commission will be able to fully deal with all of the matters in his complaint. While the complaint deals with harassment, there are also allegations of adverse differential treatment within the workplace. If the harassment complaint process were still available to him, it is not clear whether it could deal with the allegations with respect to the transfer occurring as a result of his marital status, for example. However, it could have dealt with much [ sic ] of the substantive allegations in the complaint. 72. Timeliness is another consideration with respect to this complaint, which is nearly four years old. It appears that both parties bear some responsibility for the delays in the process. In 2010, the complainant did attempt to speak to his superiors about his allegations, but he did not provide them with the requested information when asked in September 2010. As outlined in paragraph 64, the respondent's process is lengthy, and there appears [ sic ] to have been delays in the processing of the complainant's grievance. 73. In terms of delay, it is not clear that the Commission's process would be more expedient than that of the respondent's internal harassment complaint process, if it were still available. Nonetheless, it appears at this stage, the Commission's process is the only remaining process available to him. 74. The complainant's human rights allegations, if proven, are serious; this is acknowledged by the respondent. The complainant has not provided any information to indicate that he is vulnerable personally, but there is a public interest in investigating allegations of racism in Canadian government institutions such as the Canadian Forces. Although the complainant describes the complaint as one of systemic discrimination, no policy or practice has been identified in the summary of complaint. There is no indication that there is or was any potential for harm to anyone if the respondent's harassment complaint process had been used. 75. The complainant suggests that that [ sic ] requiring him to engage in the harassment complaint process is unfair. Although the complainant's scepticism may be understandable given the results of the IA decision, there is no indication that the respondent's alternate redress procedures overall are so flawed that it could not have dealt with the complainant's allegations within its harassment complaint process. In this instance, the Final Authority made recommendations in order to specifically address the issues that occurred previously, including any potential unfairness and the issue of delay. 76. Having regard for all of the above, it appears that the complainant had another review procedure, the harassment complaint process, reasonably available to him after the Final Authority decision in 2013.", "On the question of whether the failure to exhaust the CF's harassment complaint process was attributable to the Applicant and not another, as set out in s 42(2) of the CHR Act , the Section 40/41 Report noted that the history of the complaint illustrated that much of the discord related to the appropriate process for addressing a complaint that raises a variety of issues such as was the case in this matter, which involved allegations of harassment, abuse of authority and adverse differential treatment.", "Ultimately, however, it concluded that: 80. The complainant acknowledged the internal harassment complaint process. After the Final Authority decision, the complainant sent in his grievance form in lieu of a harassment complaint. In his letter to the complainant dated May 22, 2013, the RO responded to the complainant to explain that the grievance form did not provide the information necessary to initiate a harassment complaint. The letter described what was required in order to file a harassment complaint and also offered assistance to the complainant. On July 19, 2013, the RO sent a follow-up letter to the complainant requesting additional information, but the complainant did not provide it. Therefore, while he acknowledged this process, he refused to participate by not providing the appropriate material in order for it to have any possibility of success. This is solely attributable to the complainant. Relevant Legislation", "The sole issue in this application for judicial review is whether the Commission's interpretation and application of s 41(1)(a) were reasonable.", "Jurisprudence has previously held that the appropriate standard of review with respect to decisions of the Commission not to deal with a complaint pursuant to s 41(1) of the CHR Act is reasonableness ( FRAGMENT_SUPPRESSED).", "Reasonableness is concerned with the existence of justification, transparency and intelligibility within the decision-making process. It is also concerned with whether the decision falls within a range of possible, acceptable outcomes which are defensible in respect of the facts and the law ( FRAGMENT_SUPPRESSED). Preliminary Issue", "A preliminary issue that has been raised by the Respondent is the admissibility of the affidavit of Nathalie Bui sworn October 24, 2014 (the Bui Affidavit). It attaches what the affiant describes as the entire record of official correspondence between the Commission and the Applicant concerning files 20101251 and 20101252.", "As noted by the Respondent, this is the Applicant's second attempt to place this material before the Court. The Applicant previously brought a motion, pursuant to Rule 317 of the Federal Courts Rules , SOR/98-106 [ FC Rules ], seeking to have the Commission's entire file produced as part of the Certified Tribunal Record (CTR).", "By Order dated November 26, 2014, Prothonotary Tabib refused that request on the basis that the Applicant was not entitled to further production, noting, in particular, that the Applicant had not identified any document that might have been considered by the Commission, or might be relevant to the determining of the issues before the Court, and which had not been included in the CTR.", "Therefore, in my view, this Court has already ruled on this issue. The conduct of this judicial review on the merits of the Applicant's claim does not serve as an appeal of the Prothonotary's decision. An appeal of a prothonotary's decision must be brought by motion to the Federal Court ( FC Rules , R 51(1), the Applicant did not bring such a motion and the time within which he was required to do so has elapsed. On that basis alone the Bui Affidavit is inadmissible and submissions made in reliance on it or in reference to it are to be disregarded.", "In any event, this Court has also previously held that material that was not before the decision-maker should generally be excluded for purposes of judicial review ( FRAGMENT_SUPPRESSED). And in this case, the Bui Affidavit does not fall under any of the recognized exceptions to this rule ( FRAGMENT_SUPPRESSED).", "The Commission listed the documentation that was reviewed when making its decision, which was contained in the CTR. Accordingly, the additional material contained in the Bui Affidavit is not relevant to the issue before this Court, being whether the Commission's Decision was reasonable based on the record that was before it.", "Further, the May 6, 2014 Section 40/41 Report specifically acknowledges that the parties had raised facts in their submissions that had occurred prior to the original Section 40/41 Report of July 6, 2011. The May 6, 2014 Section 40/41 Report stated that, because the Commission had already dealt with the alternative redress procedures available to the Applicant prior to July 2011, its current analysis would be based only on information arising after the Commission's original decision (para 61). As seen from the analysis contained in the May 6, 2014 Section 40/41 Report, prior facts which were set out in the parties' submissions do not constitute a part of the analysis.", "For all of these reasons, it is my view that the Bui Affidavit is inadmissible. Accordingly, the Applicant's submissions made in reliance on or in reference to that affidavit have not been considered in this judicial review. Position of the Parties", "The Applicant's position is that he has spent years attempting to satisfy the CF grievance system and acted in good faith including through a CF led biased and fatally flawed investigation and ensuing administrative investigation report. He characterises as cyclical, absurd and unreasonable the suggestion that to satisfy all reasonable steps he should now engage the harassment procedure and, if unsatisfied, pursue a new grievance, potentially all the way to judicial review. That process would create an endless procedural loop that would prevent the Applicant from exercising his right to complain under the CHR Act . Accordingly, the Commission unreasonably concluded that the CF's internal process was reasonably available to him.", "The Applicant further submits that the Commission ought to have accepted his complaint. The CF grievance process was exhausted because the Chief of the Defence Staff failed to make a decision on the grievance and instead passed it off, requiring the harassment procedures to be initiated. Further, the CF has demonstrated an unwillingness and an inability to handle this complex matter.", "The Respondent submits that a review of the reasonableness of the Commission's Decision should include an examination of some or all of the relevant factors ( FRAGMENT_SUPPRESSED), including any delay attributable to the Applicant. It should not include the Applicant's subjective and unsubstantiated view that the harassment process is not reasonably available and inadequate because the Applicant has never engaged the process.", "The Respondent further submits that the Commission's Decision that the Applicant ought to have used the appropriate CF internal process, which was reasonably available, falls within a range of possible acceptable outcomes. Any circularity in the process, as alleged by the Applicant, is purely attributable to his own refusal to utilize the process that the CF, the FA and the Commission all found to be the appropriate avenue for redress. Analysis", "In this matter the Commission adopted the recommendations made in the Section 40/41 Report, which, in that circumstance, constitute the reasons of the Commission ( FRAGMENT_SUPPRESSED). The Applicant takes no issue with the Commission's form of response.", "The Section 40/41 Report stated that s 41(1)(a) and s 42 together mean that the Commission can decide not to deal with a complaint under s 41(1)(a) if it finds that the complainant chose not to finish another process that was reasonably available to him or her. This is not an unreasonable interpretation of these provisions. In FRAGMENT_SUPPRESSED", "The Section 40/41 Report then listed factors that may be considered when deciding under s 41(1)(a) whether or not a complainant should keep using the other complaint or grievance process. These were: why the parties did not finish the other process; how much longer that process was likely to take; whether there is new information in that regard; and whether there is anything other than the amount of time the other process may take, such as vulnerability arising from the complainant's current situation or a risk of harm to any participant, that makes it not \"reasonably available\" to the complainant such that the Commission should deal with the complaint. The Section 40/41 Report also summarized the CF's submissions of February 20, 2014, and the Applicant's submissions of February 21, 2014 and April 10, 2013.", "The question of whether the harassment complaint process was reasonably available to the Applicant is largely a factual question. In my view, there is no question that it was available. The only question being was it reasonably so.", "In July 2011 the Commission decided not to deal with the complaint at that time under s 41(1)(a); instead, it asked the Applicant to use another complaint or grievance process first. At the end of that process the complainant could come back to the Commission and ask it to reactivate the complaint.", "Given this, the Applicant then filed a grievance pursuant to s 29 of the National Defence Act , RSC 1985, c N-5. The process thus engaged is set out in the Queen's Regulations and Orders for the Canadian Forces (QROCF) (online:  ). This envisions consideration and determination of the grievance by the IA (s 7.15). If the complainant is not satisfied with the IA decision, he or she may request that the matter be referred to the FA (s 7.18). The FA for all grievances is the Chief of the Defence Staff ( National Defence Act , s 29) or his delegate (QROCF, s 7.17). Pursuant to s 29.12 of the National Defence Act, the Chief of the Defence Staff shall refer every grievance that is of a type prescribed by regulation to the Grievances Committee for its findings and recommendations before the Chief of the Defence Staff considers and determines the grievance. These types of grievances are set out in s 7.21 of the QROCF. They include any decision of the Chief of the Defence Staff in respect of a particular officer or non-commissioned officer.", "In March 2012 the IA decision was issued. As the Applicant was unsatisfied with that decision, in May 2012 he sought an FA decision. As seen from the March 4, 2013 letter from the Chief of the Defence Staff, the grievance was submitted to the CFGB which conducted an independent review of the grievance and provided findings and recommendations which were disclosed to the Applicant.", "The Chief of the Defence Staff stated that the CFGB had analysed each of the contentions in the grievance file and found that the administrative investigation on which the IA had based its decision was \"fatally flawed\" on several levels.", "He also stated: The CF's policies on racist conduct and on harassment prevention and resolution are outlined in Canadian Forces Administrative Order 19-43 (Racist Conduct) and Defence Administrative Order and Directive (DAOD) 5012-0 (Harassment Prevention and Resolution). Both are unequivocal in their condemnation of any conduct that causes offence or harm to any person through acts, comments, or displays that demean, belittle, and cause personal humiliation or embarrassment, and any act of intimidation or threat. As much as we in the CF would like to think that we are immune from such behaviours, as a microcosm of the society that we serve, we would be very naïve to think that we, as an organization, would not be subject to instances of this insidious conduct. Accordingly, the CF has a comprehensive framework to deal with such instances in the form of the Harassment Prevention and Resolution Guidelines . The CF policy and guidelines flow directly from and are consistent with the Treasury Board's Policy on Harassment Prevention and Resolution in the Workplace and go beyond the requirements of the Canadian Human Rights Act in their scope.", "The Chief of the Defence Staff expressed concern that the Applicant had chosen not to avail himself of the CF harassment process. His lack of cooperation by failing to fulfil the obligation to provide full written details of his complaint was an impediment to the fair, impartial and expedient harassment investigation and contributed to the \"procedural morass\" that followed. The Chief of the Defence Staff went on to say that there had been important evidentiary omissions and procedural errors that compromised the integrity of the administrative investigation process resulting in what the CFGB had rightly referred to as a fatally flawed administrative investigation; consequently, the IA decision that was based on that investigation was equally flawed.", "Although the Chief of the Defence Staff had identified his role as being to determine whether the harassment occurred and whether the Applicant was a victim of mistreatment at the hands of a superior officer, he concluded that in light of the CFGB findings, he could not render a properly informed decision. He therefore decided to order a new harassment investigation.", "The Chief of the Defence Staff's reference to a lack of clarity as to the order in which the CF harassment and grievance processes are to be utilized is of note. He stated that the CFGB had highlighted that there was lack of clear policy direction regarding the relationship between the harassment and grievance processes. DAOD 2017-1 states: The following are situations in which other complaint and mechanisms should be used prior to proceeding with a grievance: • a complaint of harassment or abuse of authority", "While s 4.10 (Coincidental Complaints and Grievances) of the Harassment Prevention and Resolution Guidelines states: If an individual decides to file a grievance on the same issue as a harassment complaint, the applicable grievance mechanism will apply and the harassment complaint will be closed.", "The Chief of the Defence Staff stated that determination of misconduct, as it pertains to harassment or acts of racism, must be ascertained through a procedurally fair and impartial investigation process, such as that outlined in the Harassment Prevention and Resolution Guidelines . The grievance process should be engaged only if the complainant feels they have been wronged as a result of the outcome of the investigation process or if there is an excessive and unjustifiable delay in handling the complaint.", "The Chief of Military Personnel (CMP), in consultation with the Assistant Deputy Minister Human Resources - Civilian, \"will harmonize the Harassment Prevention and Resolution Guidelines with DAOD 2017-1 so that it is very clear that harassment complaints or grievances containing allegations of harassment must be dealt with first under the harassment policy and only after a proper harassment investigation has been completed\".", "The Chief of the Defence Staff decided that:", "am not prepared to grant the redress that you seek. Yet, I am troubled by your allegations and most particularly, by the lack of a proper investigation into their validity. As I mentioned, the CF takes this issue very seriously and consequently, it is imperative that you initiate the harassment investigation process by submitting a formal harassment complaint to the CO of 4 CFMCU. You have 90 days from the receipt of this letter to make your submission. The CO of 4 CFMCU shall, upon receipt of your harassment complaint, initiate a harassment investigation into the allegations therein. Let me be very clear on this point: an investigation will not commence until there is a formal harassment complaint submitted to the CO of 4 CFMCU and you commit, in writing, that you will fully participate in such an investigation to ensure that a thorough and impartial analysis may determine the legitimacy of your allegations. If you do not respond within 90 days, there will be no further action taken on your alleged complaints. Given the complexities of this case, compounded by the length of time that has elapsed since the alleged incidents, the CO of 4 CFMCU might consider engaging a private consulting firm to assist with the investigation ... When the investigation has been completed, you may grieve its outcome if the conclusions and recommendations are not to your satisfaction.", "In my view, regardless of the factual background prior to the Chief of the Defence Staff's decision and regardless of the lack of clarity as to the order in which harassment complaints or grievances containing allegations of harassment were to be dealt with and any delay or confusion that this may have previously caused, as of March 13, 2013, the Applicant was on clear notice of what was required of him, and that was that he initiate a harassment complaint within 90 days. He was also aware that if he failed to do so, there would be no further action taken with respect to the allegations.", "The CF's submission, as summarized by the Section 40/41 Report, stated that on May 14, 2013 it received from the Applicant a photocopy of his original grievance, not a formal harassment complaint as required by the Chief of the Defence Staff. On May 22, 2013, the RO sent the Applicant a letter requesting that he redraft his submission, explain each allegation specifically and provide any information that would enable the harassment investigation to be thorough and complete. The RO also notified the Applicant's current CO in order to ensure that an assistant could be appointed for the Applicant to help him with the process. The Applicant did not respond to the RO's request. The RO sent another letter to the Applicant on July 19, 2013 requesting an update on the preparation of this harassment complaint but again did not receive a response.", "The Section 40/41 Report noted the other factors that were considered in reaching its recommendation, including the attempt of the Chief of the Defence Staff to address the procedural fairness and other concerns by having the matter dealt with by a formal harassment complaint, with an outside firm being hired to investigate. Further, as to timeliness, that both parties bore some responsibility for the prior delays in the process and that it was not clear that the Commission's process would be more expedient than the CF's harassment policy, if it were still available to the Applicant. However, at this stage, the Commission's process was the only one still available to the Applicant. Further, that the Applicant had provided no information to indicate that he was personally vulnerable.", "The Applicant's position is that the harassment complaint process as proposed by the Chief of the Defence Staff cannot be considered to be reasonably available. The Applicant submits that there has already been a considerable delay, that he commenced the grievance process and that, at the end of that process, rather than rendering a decision, the Chief of the Defence Staff instead attempted to effect a harassment investigation. This is \"cyclical\" because he was required to commence the grievance process, which now requires a harassment investigation, which in turn can be grieved, and that this has the potential of becoming an endless loop.", "In my view, this argument cannot succeed. Although the CFGB's report was not in the CTR, it is clear from the Chief of the Defence Staff's letter that there was a lack of clarity regarding the relationship between the harassment and grievance processes. This may well have contributed to the prior delay and procedural missteps in this matter. And, unless the relationship is clarified, it may have the same impact on other matters in the future.", "Regardless, in this case, the decision by the Chief of the Defence Staff had the effect of resetting the clock. While it is true that a grievance from the conclusions and recommendations of the harassment investigation would have been available to the Applicant, this did not mean that an endless procedural circle would follow. The problem identified in the original grievance process initiated by the Applicant was that there was insufficient information to deal with the harassment complaint. This led to the Chief of the Defence Staff's decision that a harassment investigation was required. Had that been carried out, any further grievance process then open to the Applicant would have been based on the findings of the harassment investigation, not the absence of one, and would not have resulted in a decision that a harassment investigation was required. Thus, this does not result in an endless cycle, nor does it establish that the process proposed by the Chief of the Defence Staff was not \"reasonably available\" to the Applicant.", "Further, the original grievance process would have been exhausted if the Applicant had participated in the proposed harassment investigation. The Applicant does not submit that it was beyond the authority of the Chief of the Defence Staff to proceed as he did, by requiring the harassment investigation. In addition, he discontinued his judicial review of the FA.", "also do not agree with the Applicant's submission that the Chief of the Defence Staff admitted that the CF is unable or unwilling to fully and objectively determine a complaint of racial discrimination when he stated the CO \"might consider engaging a private consulting firm to assist the investigation\" in his letter given the complexity of the case, compounded by the length of time that had elapsed since the alleged incidents. A suggestion that an independent third party with experience in the investigation of harassment complaints could be utilized, if anything, would seem to suggest the incorporation of an additional measure to ensure procedural fairness in the process.", "The Applicant also submits that the Chief of the Defence Staff's statement quoted at paragraph 42 above is an acknowledgment that racism exists within the CF, at least to some degree, and that his inaction to combat racism may mean that it will only continue to exist. However, read in whole, the statement clearly expresses that the CF Harassment Prevention Resolution Guidelines are intended to deal with any instance of racism and harassment.", "As to systemic discrimination, the Section 40/41 Report stated that although the Applicant described the complaint as one of systemic discrimination, no policy or practice had been identified in the summary of complaint. A review of the complaint confirms that the complaint clearly pertained to the Applicant's CO and made no allegation of systemic discrimination. At the hearing before me, the Applicant confirmed that this was the case and that systemic discrimination was therefore not pursued as an issue in this matter.", "In summary, the Commission considered the factual basis of this matter and found that there was a review procedure that was reasonably available to the Applicant and that had not been exhausted by the Applicant prior to the expiry of the 90 day deadline set by the Chief of the Defence Staff. Based on the record, this finding was reasonably open to it.", "The Commission also recognised that this finding would leave the Applicant without recourse. However, as noted by the Respondent, this Court has held that the reasonableness of a decision is not affected by the absence of the \"safety net\" of renewed access to the Commission ( FRAGMENT_SUPPRESSED). In this case, while the CF review process itself is no longer available to the Applicant, nor is there a safety net, the Commission recognized this outcome when making its decision.", "In the circumstance of this matter, the Commission's finding that the Applicant was aware of, but refused to participate in, the CF's internal harassment complaint process and that the failure to exhaust the procedure was solely attributable to the Applicant is to be afforded deference ( FRAGMENT_SUPPRESSED) and also fell within a range of possible acceptable outcomes defensible in respect of the facts and law ( FRAGMENT_SUPPRESSED).", "For these reasons the application is dismissed. The parties have agreed to costs in the amount of $2,500.00, accordingly, the Respondent shall have its costs in that amount. JUDGMENT", "THIS COURT'S JUDGMENT is that 1. The application for judicial review is dismissed. 2. The Respondent shall have its costs in the amount of $2,500.00. Application dismissed. Editor: Jana A. Andersen/nmg [End of document]"], "c_paras": ["Russell, J. : This is an application under s. 18.1 of the Federal Courts Act , RSC 1985, c F-7 for judicial review of a decision of the Canadian Human Rights Commission [Commission], dated May 7, 2014 [Decision], which decided not to deal with the Applicant's complaint pursuant to s. 41(1)(d) of the Canadian Human Rights Act , RSC 1985, c H-6 [Act].", "The Applicant started working as a Heavy Equipment Operator for Canadian National Railway Company [CN] in August 1981. He was a member of a bargaining unit represented by the National Automobile, Aerospace, Transportation and General Workers Union of Canada (CAW-Canada) [ Union].", "On December 19, 2012, the Applicant's employment was terminated because he refused to attend a medical assessment to determine his fitness for work. CN requested the medical assessment because the Applicant's position was a safety sensitive position and there were concerns regarding his behaviour. The Applicant says that he refused to attend the medical assessment because he was receiving treatment for drug dependency and believed that was sufficient to address CN's concerns.", "On January 8, 2013, the Union filed a grievance to contest the Applicant's termination.", "On February 14, 2013, the Union closed the Applicant's file. The Union said that the Applicant had not responded to their requests for information and it \"was not in a position to advance the matter with the limited information at hand.\"", "In March 2013, the Applicant submitted a complaint to the Commission. He alleged that CN had discriminated against him on the ground of disability by terminating his employment contrary to s. 7 of the Act. The Commission decided not to deal with the complaint pursuant to s. 41(1)(a) because the Applicant had not exhausted the Respondent's grievance process.", "On April 19, 2013, CN denied the Union's grievance. The Union did not refer the grievance to arbitration.", "On October 1, 2013, the Applicant submitted another complaint to the Commission. He alleged that, again, CN had discriminated against him on the ground of disability by terminating his employment contrary to s. 7 of the Act.", "On October 29, 2013, the Applicant was advised that the Commission would be preparing a s. 40/41 report to determine whether it should deal with his complaint. The Applicant was invited to prepare a letter stating his position on whether the Commission should not deal with the issues because \"the human rights issues in this complaint may have already been dealt with through another process.\" Counsel for the Applicant made submissions to the Commission both in advance of the preparation of the report and after being provided a copy of the s. 40/41 report [Report].", "On May 7, 2014, the Commission decided not to deal with the Applicant's complaint pursuant to s. 41(1)(d) of the Act. The Commission adopted the Report's conclusions and decided that the complaint was vexatious under s. 41(1)(d) of the Act (Applicant's Record at 48): The complainant's human rights allegations have been addressed by an alternate decision maker with authority to consider human rights issues. The allegations raised in the complaint before the Commission are the same as those addressed in the final level grievance response. Given that the alternate decision-maker dealt with the human rights issues raised in this complaint, and that process was fair, the Commission must respect the finality of that decision and should not deal with this complaint. It is therefore plain and obvious that this complaint is vexatious within the meaning of section 41(1)(d) of the Act.", "The Applicant raises the following issues in this application: 1. Whether the Commission unreasonably refused to exercise its jurisdiction; 2. Whether the Commission erred in law by: a) Unreasonably finding the Applicant's complaint to be vexatious; or, b) Having found the complaint to be vexatious, unreasonably ignoring that justice required it to deal with the complaint anyway; and, 3. Whether the Commission unreasonably based its decision on erroneous findings of fact made without regard to the material before it.", "The Supreme Court of Canada in Dunsmuir v New Brunswick , 2008 SCC 9 [ Dunsmuir ] held that a standard of review analysis need not be conducted in every instance. Instead, where the standard of review applicable to a particular question before the court is settled in a satisfactory manner by past jurisprudence, the reviewing court may adopt that standard of review. Only where this search proves fruitless, or where the relevant precedents appear to be inconsistent with new developments in the common law principles of judicial review, must the reviewing court undertake a consideration of the four factors comprising the standard of review analysis: Agraira v Canada (Public Safety and Emergency Preparedness) , 2013 SCC 36 at para 48.", "The Applicant submits that decisions under s. 41(1)(d) of the Human Rights Act are reviewed on a standard of reasonableness: Chan v Canada (Attorney General) , 2010 FC 1232 [ Chan ]. The Respondent submits that the Commission's decision not to deal with a complaint under s. 41 of the Human Rights Act is a discretionary decision reviewable on a standard of reasonableness: Exeter v Canada (Attorney General) , 2011 FC 86 at para 19, aff'd 2012 FCA 119 at para 6 [ Exeter ]; Morin v Canada (Attorney General) , 2007 FC 1355 at para 25, aff'd 2008 FCA 269.", "All of the issues question the reasonableness of the Commission's decision to not deal with the complaint. The Court agrees that these decisions are reviewable on a standard of reasonableness: Chan , above, at para 15; Exeter , above, at para 6.", "When reviewing a decision on the standard of reasonableness, the analysis will be concerned with \"the existence of justification, transparency and intelligibility within the decision-making process [and also with] whether the decision falls within a range of possible, acceptable outcomes which are defensible in respect of the facts and law\": see Dunsmuir , above, at para 47; Canada (Citizenship and Immigration) v Khosa , 2009 SCC 12 at para 59. Put another way, the Court should intervene only if the Decision was unreasonable in the sense that it falls outside the \"range of possible, acceptable outcomes which are defensible in respect of the facts and law.\"", "A. Applicant", "The Applicant submits that the Commission unreasonably refused to exercise its jurisdiction. The Applicant concedes that the Commission is entitled to adopt the Report for its reasons: Chan , above, at paras 39-40. However, the Applicant distinguishes the present proceeding from the Chan case on two grounds. First, the Commission's adoption of the Report was inadequate because it fails to show that the Commission considered the submissions before it and fails to recognize that the human rights issues were not considered in the grievance process: Vancouver International Airport Authority v Public Service Alliance of Canada , 2010 FCA 158. Second, the internal grievance process does not constitute a proper decision-maker. The grievance process is not an independent arbitrator and it failed to provide reasons for its decision on the human rights issues.", "The Applicant characterizes the grievance process as an internal negotiation between the Union and CN. If the Commission does not deal with the complaint then he says the Union's Decision to not refer the grievance to arbitration will have denied him the ability to have his human rights issue considered by a decision-maker. The Union's Decision to not proceed to arbitration was based, in part, on the Applicant's refusal to cooperate but the Applicant says that the nature of his disability carries the need for reasonable flexibility regarding deadlines and expectations. The Union's Decision was also based on other factors including time, money and resources.", "In the alternative, if the internal grievance process constituted a decision-maker, the Decision is unreasonable because the grievance did not address the human rights issues. It referenced \"drug dependency\" and concluded there was insufficient evidence.", "If the complaint was correctly deemed vexatious, then the Applicant submits that the Commission erred in law by ignoring that justice required the Commission to deal with the complaint anyway. The Decision says the internal grievance process was fair but fails to consider the Applicant's reply submissions.", "Finally, the Decision is unreasonable because it relies on the erroneous finding that the Applicant's human rights issues were already addressed by a decision-maker. B. Respondent", "The Respondent submits that it was reasonable for the Commission to refuse to deal with the complaint. The Report constitutes the reasons for the Decision: Canada (Attorney General) v Sketchley , 2005 FCA 404 at para 37 [ Sketchley ]; Bergeron v Canada (Attorney General) , 2013 FC 301 at paras 28-29 [ Bergeron ]. The Applicant had the opportunity to address the human rights issues through the Union but he failed to cooperate with the search for accommodation. The Commission may refuse to deal with a complaint if it is obvious that the complaint cannot succeed. A complainant who refuses to collaborate in the search for accommodation will have his or her complaint dismissed: Central Okanogan School District No 23 v Renaud , [1992] 2 SCR 970.", "The Commission's Decision to refuse to deal with the complaint is also in accordance with Supreme Court of Canada jurisprudence regarding the importance of permitting administrative tribunals to curb abuse of process: British Columbia (Workers' Compensation Board) v Figliola , 2011 SCC 52. It would be an abuse of process to advance a human rights complaint where the complainant has failed to cooperate with their union to have the same human rights issues addressed.", "Contrary to the Applicant's submissions, s. 41(1)(d) does not require that a decision be made by an arbitrator. The Commission is granted great latitude in exercising its discretion and assessing the appropriate factors in performing its screening function: Sketchley , above, at para 38; Bergeron , above, at para 39. Further, the Federal Court has held that s. 41(1)(d) may apply in situations where a union has decided not to pursue a grievance to arbitration: Bergeron , above, at para 38. There is also no evidence that those who decided the Applicant's grievances were not impartial: Bergeron , above, at para 43.", "The Report shows that the Investigator turned her mind to the outcome of the grievance process, the Applicant's allegations relating to substance abuse, and the question of reasonable accommodation. The Commission reasonably concluded that the allegations raised in the complaint had already been addressed in the grievance process and that the grievance process was fair.", "The Applicant raises three (3) grounds for reviewable error but, in the end, they all come back to the issue of the Applicant's own failure to cooperate in the grievance process. Essentially, the Commission came to the conclusion that the Applicant's complaint was vexatious under s. 41(1)(d) of the Act because the Applicant's human rights allegations had already been addressed by the grievance process.", "As the Report found, the Applicant's Union representative filed a grievance on his behalf that raised the same human rights issues as those in the complaint to the Commission. The Union had to close out its grievance file because the Applicant would not cooperate with its attempts at obtaining accommodation for him. The Union concluded that it could not advance the grievance to arbitration because, given Applicant's failure to cooperate in providing the information requested and required for the grievance process, there was insufficient information to advance the matter. In the end, the grievance process was exhausted without going to arbitration because the Applicant failed to cooperate. This was the final decision in the grievance process.", "The Applicant attempted to convince the Commission, and he has attempted to convince this Court, that his disability prevented him from providing the materials and cooperation required by the grievance process. In his submissions to the Commission, he alleged as follows (CTR at 14): Given the nature of [the Applicant's] disability, it follows that it would be logical for the Company to have been in closer contact with the Union in order to determine what the correct situation, and prognosis, for [the Applicant] was. It is understandable that an individual with a disability would encounter difficulty in navigating deadlines without reasonable assistance, and it was further understandable that [the Applicant] mistakenly believed the matter was being dealt with by his Union and his doctor.", "There was no evidence before the Commission, and there is none before me, to support this bare allegation that the Applicant had difficulties with deadlines and mistaken beliefs because of his disability. The Applicant simply expected the Commission, and now asks the Court, to draw an inference to this effect from the nature of his disability which is drug dependency. It is noticeable that, in the affidavit he has filed with this application, the Applicant says nothing about difficulties with deadlines and mistaken beliefs. Further, his evidence before me clearly indicates the considerable lengths to which the Union went to make clear to the Applicant what was required of him and to encourage him to comply. The letter of April 4, 2013 to the Applicant from Mr. Robert Fitzgerald, the Union's National Representative, sets out the whole picture: To date, none of this information has been provided. The Union only has two pieces of medical documentation. One dated January 24, 2013 stating that you will be seeing an addiction counsellor, but no confirmation that you did. The second one verified that you do not have any disability as it relates to psychiatric issues. Although your doctor invited us to follow up with him providing we had the necessary medical release to do so, you failed to return the release form that the Regional Representative provided to you, enclosed with his letter of February 01, 2013. There have been literally hundreds of phone calls between you and the Union at different levels. However, you have not acknowledged the Union's request for information. You have abated the Union's request in must the same way you have declined to cooperate with the Company. In our opinion, the negative connotation of your actions would not be lost on an Arbitrator. At some point you did advise the Company that you had an addiction problem and that you were seeking help for such. However, there is no evidence that you have been diagnosed with such an addiction nor is there any evidence to show that you are being treated for such. As we said earlier, there is only two pieces of medical documentation on file and neither provide a diagnoses or address treatment. If in fact there was a clinical diagnosis of addiction, treatment and rehabilitation, the Company may well have been obligated to provide accommodation. However, with such an obligation, there also comes an onus on the employees to cooperate with the efforts to accommodate. It was put this way by the arbitrator in CROADR case 3354: The Arbitrator must agree. As confirmed by the Supreme Court of Canada in Central Okanogan School District No. 23 v. Renaud,", "2 S.C.R. 970, the obligation of accommodation involves the cooperative participation of the employer, the trade union and the employee . That was reflected in an award of this Office in CROA 3173 : The Arbitrator is satisfied that the approach adopted by the Company is in keeping with its obligations under the Canadian Human Rights Act . It now seems well-established that when an employee seeks accommodation by reason of a status that is protected under the Canadian Human Rights Act, it is incumbent upon the employee concerned to contribute positively to the process , and to accept an offer of reasonable accommodation, even though it might not be the specific accommodation which the employee would prefer. That is reflected, in part, in the decision of the Supreme Court of Canada in Central Okanogan School District No. 23 v. Renaud,", "2 S.C.R. 970. In that decision, for a unanimous court, Sopinkla J. wrote as follows : To facilitate the search for an accommodation, the complainant must do his or her part as well . Concomitant with a search for reasonable accommodation is a duty to facilitate the search for such an accommodation. Thus in determining whether the duty of accommodation has been fulfilled the conduct of the complainant must be considered. [Emphasis in original]", "On the basis of the record that was before the Commission, and that is before me, the only possible inference is that the Union made every effort to advance the Applicant's grievance but had to abandon the process at step III because of the Applicant's refusal to provide the necessary information, a refusal that has not been linked to his alleged disability. The Commission deals with this matter extensively in the Decision by referring to Mr. Fitzgerald's letter and the Step III Grievance Response dated April 19, 2013. The Applicant provided nothing to counter the information regarding his non-cooperation. It has to be remembered that it was the Applicant who provided the letter from Mr. Fitzgerald so that he was well-aware of what it said about him, and it also has to be borne in mind that his non-cooperation is evidenced by his own Union who had supported him in the grievance process. There was nothing to suggest that the Applicant's failure to cooperate had anything to do with his disability.", "It is also noteworthy that the Canadian Industrial Relations Board came to a similar conclusion when the Applicant alleged a violation of s. 37 of the Canada Labour Code , and alleged that the Union breached its duty of fair representation by failing to properly represent him when it decided not to proceed further with his grievance ( Mulligan v National Automobile, Aerospace, Transportation and General Workers Union of Canada (CAW-Canada) (31 July 2013), 29997-C (CIRB):", "In this case, the complainant requests that the Board hold a hearing. Section 16.1 of the Code provides that the Board may decide any matter before it without holding an oral hearing. Having reviewed all of the material on file, the Board is satisfied that the documentation before it is sufficient for it to decide the matter without holding an oral hearing. As mentioned above, the complainant alleges that the union acted in an arbitrary manner and in bad faith when it did not properly investigate his grievance, did not contact him and did not seek the proper information from his doctors and counsellor. The complainant also alleges that the union violated his rights with respect to article 23.2 of the collective agreement. Section 37 of the Code reads as follows: 37. A trade union or representative of a trade union that is the bargaining agent for a bargaining unit shall not act in a manner that is arbitrary, discriminatory or in bad faith in the representation of any of the employees in the unit with respect to their rights under the collective agreement that is applicable to them. The Board's role in the context of a duty of fair representation complaint is to examine the union's conduct in handling the employee's grievance (see Bugay , 1999 CIRB 45). A section 37 complaint cannot serve to appeal a union's decision not to refer a grievance to arbitration, or to assess the merits of the grievance, but it is used to assess how the union handled the grievance (see Presseault , 2001 CIRB 138). In a complaint under section 37, the complainant bears the onus of presenting evidence that is sufficient to raise a presumption that the union has breached its duty of fair representation. The Board will normally find that the union has fulfilled its duty of fair representation if it has investigated the circumstances, considered the merits of the grievance, made a reasoned judgment about whether to pursue the issue, and if it advised the employee of the reason for its ultimate decision not to proceed any further. The duty of a member to cooperate with his union is described in the following passage from McRaeJackson , 2004 CIRB 290:", "The union's duty of fair representation is predicated on the requirement that employees take the necessary steps to protect their own interests. Employees must make the union aware of potential grievances and ask the union to act on their behalf within the time limits provided in the collective agreement. They must cooperate with their union throughout the grievance procedure, for example by providing the union with the information necessary to investigate a grievance, by attending any medical examinations or other assessments. The evidence on file indicates that the union filed a grievance on behalf of the complainant, processed the grievance to step three of the grievance procedure, sent several letters to the complainant seeking medical information and had numerous telephone conversation with the complainant, with limited success in getting the information needed to further his case. In the Board's opinion, the complainant did not provide any evidence of wrongdoing by the union. The documentation submitted indicates that the complainant brought his termination upon himself by not submitting the information requested by the union. Failure by the complainant to take such action, along with his refusal to cooperate with the union, leads the Board to conclude that the union did not act in an arbitrary manner or in bad faith. Having reviewed the facts submitted, the Board finds that the complainant did not provide sufficient facts to establish that the union has violated its duty of fair representation. For the above reasons, the complaint is dismissed.", "This decision by the Canadian Industrial Relations Board was not before the Commission, but it confirms the Commission's conclusions that the Applicant is the one who, for no apparent reason, thwarted the grievance process that the Commission had earlier told him he had to exhaust before bringing his complaint to the Commission.", "As the Report makes clear, all of the Applicant's submissions were considered including the \"issue of consent and ongoing substance use\" that he claims was not addressed by the Respondent, as well as the correspondence from the Applicant's doctor and personnel in the Respondent's Employee Assistance Program.", "can find no reviewable error in the Commission's Decision (which includes the Report), which ably sets out the relevant facts and the governing jurisprudence. This is simply a case where the Applicant, for no apparent reason, refused to cooperate in the grievance process that could have dealt with his human rights issues and left his Union with no alternative but to close out the file.", "The Commission provides full reasons as to why the complaint was vexatious and why justice did not require the Commission to deal with the complaint.", "Subsection 41(1)(d) of the Act does not require a decision by a grievance arbitrator. As Justice Zinn pointed out in Bergeron , above:", "The jurisprudence is clear that the Commission is to be afforded great latitude in exercising its judgment and in assessing the appropriate factors when considering the application of paragraph 41(1)(d) of the CHRA and performing this \"screening function:\" See, e.g., Sketchley at para 38.", "Bergeron , above, makes it clear that s. 41(1)(d) of the Act may apply in situations where a union has decided not to pursue a grievance to arbitration. In the present case, as the Union letter makes clear, the Applicant refused, for no reason that is established, to engage in a grievance process that could have provided him with accommodation and arbitration and that could have dealt with his human rights issues. The Union makes it clear that his failure to cooperate meant that there was no point in proceeding to arbitration. Having failed to exhaust a grievance process that could have provided him with the remedy he sought before the Commission, the Applicant then filed his complaint with the Commission. The Applicant failed to show that his complaint could not have reasonably been dealt with by the grievance process. The Commission's Decision should not be disturbed.", "The Commission's Decision is transparent, intelligible and justifiable. I can find no reviewable error. It falls within the range of possible acceptable outcomes which are defensible in respect of the facts and the law. JUDGMENT", "THIS COURT'S JUDGMENT is that 1. This application is dismissed with costs to the Respondent. Application dismissed. Editor: J. Danielle King/bk [End of document]"], "label": 1}
# task1=dict()
# task1['guid']='1839d9b_d6f26064d6f489ae6e1a681139e58eb7223837a0'
# task1['q_paras']=['Detection of CDE as active WM is platform-spe...','SWT code currently determines the active deskt...']
# task1['c_paras']=['quert_1','query_2']
# task1['label']=1
# task1
# with open('case_para_sample.json','w',encoding='utf-8') as f_out:
#     for i in range(4):
#         f_out.write(str(task1)+'\n')



# 模型需要的输入样例 下面进行模型的测试
{
	"guid": "queryID_docID",
	"q_paras": [...], // a list of paragraphs in query case,
	"c_paras": [...], // a list of parameters in candidate case,
	"label": 0, // 0 or 1, denote the relevance
 }
 
{
	"guid": "queryID_docID",
	"res": [[],...,[]], // N * 768, result of BertPoolOutMax,
	"label": 0, // 0 or 1, denote the relevance
}



import configparser
import os
import functools


class ConfigParser:
    def __init__(self, *args, **params):
        self.default_config = configparser.RawConfigParser(*args, **params)
        self.local_config = configparser.RawConfigParser(*args, **params)
        self.config = configparser.RawConfigParser(*args, **params)

    def read(self, filenames, encoding=None):
        if os.path.exists("config/default_local.config"):
            self.local_config.read("config/default_local.config", encoding=encoding)
        else:
            self.local_config.read("config/default.config", encoding=encoding)

        self.default_config.read("config/default.config", encoding=encoding)
        self.config.read(filenames, encoding=encoding)


def _build_func(func_name):
    @functools.wraps(getattr(configparser.RawConfigParser, func_name))
    def func(self, *args, **kwargs):
        try:
            return getattr(self.config, func_name)(*args, **kwargs)
        except Exception as e:
            try:
                return getattr(self.local_config, func_name)(*args, **kwargs)
            except Exception as e:
                return getattr(self.default_config, func_name)(*args, **kwargs)

    return func


def create_config(path):
    for func_name in dir(configparser.RawConfigParser):
        if not func_name.startswith('_') and func_name != "read":
            setattr(ConfigParser, func_name, _build_func(func_name))

    config = ConfigParser()
    config.read(path)

    return config
config = create_config('/home/hdj/BERT-PLI-IJCAI2020/config/nlp/BertPoint.config')



for key,val in config.items():
    for k,v in val.items():
        print(k,v)
    print(key,val,type(val))

import torch.nn as nn
import torch.nn.functional as F
from pytorch_pretrained_bert import BertModel
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AdamW
import torch

class BertPoolOutMax(nn.Module):
    def __init__(self, config, gpu_list, *args, **params):
        super(BertPoolOutMax, self).__init__()
        self.max_para_c = config.getint('model', 'max_para_c')
        self.max_para_q = config.getint('model', 'max_para_q')
        self.step = config.getint('model', 'step')
        self.max_len = config.getint("data", "max_seq_length")
#         self.bert = BertModel.from_pretrained(config.get("model", "bert_path"))
        self.bert = RobertaModel.from_pretrained("microsoft/codebert-base")
        # self.maxpool = nn.MaxPool1d(kernel_size=self.max_para_c)
        self.maxpool = nn.MaxPool2d(kernel_size=(1, self.max_para_c))

    def init_multi_gpu(self, device, config, *args, **params):
        self.bert = nn.DataParallel(self.bert, device_ids=device)

    def forward(self, data, config, gpu_list, acc_result, mode):
        input_ids, attention_mask, token_type_ids = data['input_ids'], data['attention_mask'], data['token_type_ids']

        with torch.no_grad():
            output = []
            for k in range(input_ids.size()[0]):
                q_lst = []
                for i in range(0, self.max_para_q, self.step):
                    # print(input_ids[k, i:i+self.step].view(-1, self.max_len).size())
                    _, lst = self.bert(input_ids[k, i:i+self.step].view(-1, self.max_len),
                                       token_type_ids=token_type_ids[k, i:i+self.step].view(-1, self.max_len),
                                       attention_mask=attention_mask[k, i:i+self.step].view(-1, self.max_len))
                    # print('before view', lst.size())
                    lst = lst.view(self.step, self.max_para_c, -1)
                    # print('after view', lst.size())
                    lst = lst.permute(2, 0, 1)
                    # print('after permute', lst.size())
                    lst = lst.unsqueeze(0)
                    # print('after unsquezze', lst.size())
                    max_out = self.maxpool(lst)
                    # print('after maxpool', max_out.size())
                    max_out = max_out.squeeze()
                    # print('after squeeze', max_out.size())
                    max_out = max_out.transpose(0, 1)
                    q_lst.extend(max_out.cpu().tolist())
                    #input('continue?')
                # print(len(q_lst))
                #exit()
                assert (len(q_lst) == self.max_para_q)
                output.append([data['guid'][k], q_lst])
            return {"output": output}
        

class BertPoint(nn.Module):
    def __init__(self, config, gpu_list, *args, **params):
        super(BertPoint, self).__init__()

        self.output_dim = config.getint("model", "output_dim")
        self.output_mode = config.get('model', 'output_mode')

        # self.bert = BertModel.from_pretrained(config.get("model", "bert_path"))
        self.bert = RobertaModel.from_pretrained("microsoft/codebert-base")
        self.fc = nn.Linear(768, self.output_dim)
        if self.output_mode == 'classification':
            self.criterion = nn.CrossEntropyLoss()
        else:
            self.criterion = nn.MSELoss()
        self.accuracy_function = init_accuracy_function(config, *args, **params)

    def init_multi_gpu(self, device, config, *args, **params):
        self.bert = nn.DataParallel(self.bert, device_ids=device)

    def forward(self, data, config, gpu_list, acc_result, mode):
        input_ids, attention_mask, token_type_ids = data['input_ids'], data['attention_mask'], data['token_type_ids']
        _, y = self.bert(input_ids,
                         # token_type_ids=token_type_ids,
                         attention_mask=attention_mask,
                            # output_all_encoded_layers=False
                         )
        y = y.view(y.size()[0], -1)

        if mode == 'test' and config.getboolean('output', 'pool_out'):
            output = []
            y = y.cpu().detach().numpy().tolist()
            for i, guid in enumerate(data['guid']):
                output.append([guid, y[i]])
            return {"output": output}

        y = self.fc(y)
        y = y.view(y.size()[0], -1)

        if "label" in data.keys():
            label = data["label"]
            loss = self.criterion(y, label.view(-1))
            acc_result = self.accuracy_function(y, label, config, acc_result)
            return {"loss": loss, "acc_result": acc_result}

        else:
            output = []
            y = y.cpu().detach().numpy().tolist()
            for i, guid in enumerate(data['guid']):
                output.append([guid, y[i]])
            return {"output": output}

def init_accuracy_function(config, *args, **params):
    name = config.get("output", "accuracy_method")
    if name in accuracy_function_dic:
        return accuracy_function_dic[name]
    else:
        raise NotImplementedError

def get_prf(res):
    # According to https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
    if res["TP"] == 0:
        if res["FP"] == 0 and res["FN"] == 0:
            precision = 1.0
            recall = 1.0
            f1 = 1.0
        else:
            precision = 0.0
            recall = 0.0
            f1 = 0.0
    else:
        precision = 1.0 * res["TP"] / (res["TP"] + res["FP"])
        recall = 1.0 * res["TP"] / (res["TP"] + res["FN"])
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1


def gen_micro_macro_result(res):
    precision = []
    recall = []
    f1 = []
    total = {"TP": 0, "FP": 0, "FN": 0, "TN": 0}
    for a in range(0, len(res)):
        total["TP"] += res[a]["TP"]
        total["FP"] += res[a]["FP"]
        total["FN"] += res[a]["FN"]
        total["TN"] += res[a]["TN"]

        p, r, f = get_prf(res[a])
        precision.append(p)
        recall.append(r)
        f1.append(f)

    micro_precision, micro_recall, micro_f1 = get_prf(total)

    macro_precision = 0
    macro_recall = 0
    macro_f1 = 0
    for a in range(0, len(f1)):
        macro_precision += precision[a]
        macro_recall += recall[a]
        macro_f1 += f1[a]

    macro_precision /= len(f1)
    macro_recall /= len(f1)
    macro_f1 /= len(f1)

    return {
        "micro_precision": round(micro_precision, 3),
        "micro_recall": round(micro_recall, 3),
        "micro_f1": round(micro_f1, 3),
        "macro_precision": round(macro_precision, 3),
        "macro_recall": round(macro_recall, 3),
        "macro_f1": round(macro_f1, 3)
    }


def null_accuracy_function(outputs, label, config, result=None):
    return None


def single_label_top1_accuracy(outputs, label, config, result=None):
    if result is None:
        result = []
    id1 = torch.max(outputs, dim=1)[1]
    # id2 = torch.max(label, dim=1)[1]
    id2 = label
    nr_classes = outputs.size(1)
    while len(result) < nr_classes:
        result.append({"TP": 0, "FN": 0, "FP": 0, "TN": 0})
    for a in range(0, len(id1)):
        # if len(result) < a:
        #    result.append({"TP": 0, "FN": 0, "FP": 0, "TN": 0})

        it_is = int(id1[a])
        should_be = int(id2[a])
        if it_is == should_be:
            result[it_is]["TP"] += 1
        else:
            result[it_is]["FP"] += 1
            result[should_be]["FN"] += 1

    return result


def multi_label_accuracy(outputs, label, config, result=None):
    if len(label[0]) != len(outputs[0]):
        raise ValueError('Input dimensions of labels and outputs must match.')

    outputs = outputs.data
    labels = label.data

    if result is None:
        result = []

    total = 0
    nr_classes = outputs.size(1)

    while len(result) < nr_classes:
        result.append({"TP": 0, "FN": 0, "FP": 0, "TN": 0})

    for i in range(nr_classes):
        outputs1 = (outputs[:, i] >= 0.5).long()
        labels1 = (labels[:, i].float() >= 0.5).long()
        total += int((labels1 * outputs1).sum())
        total += int(((1 - labels1) * (1 - outputs1)).sum())

        if result is None:
            continue

        # if len(result) < i:
        #    result.append({"TP": 0, "FN": 0, "FP": 0, "TN": 0})

        result[i]["TP"] += int((labels1 * outputs1).sum())
        result[i]["FN"] += int((labels1 * (1 - outputs1)).sum())
        result[i]["FP"] += int(((1 - labels1) * outputs1).sum())
        result[i]["TN"] += int(((1 - labels1) * (1 - outputs1)).sum())

    return result


def single_label_top2_accuracy(outputs, label, config, result=None):
    raise NotImplementedError
    # still bug here

    if result is None:
        result = []
        # print(label)

    id1 = torch.max(outputs, dim=1)[1]
    # id2 = torch.max(label, dim=1)[1]
    id2 = label
    nr_classes = outputs.size(1)
    while len(result) < nr_classes:
        result.append({"TP": 0, "FN": 0, "FP": 0, "TN": 0})
    for a in range(0, len(id1)):
        # if len(result) < a:
        #    result.append({"TP": 0, "FN": 0, "FP": 0, "TN": 0})

        it_is = int(id1[a])
        should_be = int(id2[a])
        if it_is == should_be:
            result[it_is]["TP"] += 1
        else:
            result[it_is]["FP"] += 1
            result[should_be]["FN"] += 1

    _, prediction = torch.topk(outputs, 2, 1, largest=True)
    prediction1 = prediction[:, 0:1]
    prediction2 = prediction[:, 1:]

    prediction1 = prediction1.view(-1)
    prediction2 = prediction2.view(-1)

    return result
accuracy_function_dic = {
    "SingleLabelTop1": single_label_top1_accuracy,
    "MultiLabel": multi_label_accuracy,
    "Null": null_accuracy_function
}

model =BertPoint(config, 0,'', "BertPoint")


model_list = {
    "BertPoint": BertPoint,
}


def get_model(model_name):
    if model_name in model_list.keys():
        return model_list[model_name]
    else:
        raise NotImplementedError
model = get_model('BertPoint')(config, 0)

parameters = torch.load('/home/hdj/BERT-PLI-IJCAI2020/output/model/task2/1_save.pkl')

def load_state_keywise(model, pretrained_dict):
    model_dict = model.state_dict()
    # print(model_dict.keys())
    # input("continue?")
    tmp_cnt = 0
    for k, v in pretrained_dict.items():
        # kk = k.replace("module.", "")
        # print('k=', k)
        # input("continue?")
        # print('kk=', kk)
        # input("continue?")
        
        if k in model_dict and v.size() == model_dict[k].size():
            model_dict[k] = v
            tmp_cnt += 1
            
            print('here',k)
        else:
            continue
    model.load_state_dict(model_dict)
    return model

model = load_state_keywise(model, parameters["model"])

# model_to_save = model.module if hasattr(model, 'module') else model
# if hasattr(model, 'module'):
#     print('这个是存在module的****')
# elif hasattr(model,'modules'):
#     print('这个是存在modules的////')
# else:
#     print('什么都么有++++')
# for key, val in model_to_save.state_dict().items():
#     print('&'*10,key)

#TODO 
parameters = torch.load('/home/hdj/BERT-PLI-IJCAI2020/output/model/task2/1.pkl')

for key,val in parameters.items():
    print(key)

if hasattr(model, 'module'):
    print('has module')
else:
    print('model')

# for key,val in model.state_dict().items():
#     print(key)

# for key in model.modules().state_dict():
#     print(key)

# model_dict = model.state_dict()
# for (key1,val1),(key2,val2) in zip(model_dict.items(),parameters['model'].items()):
#     print(key1,' *** ',key2,' // ',key1.endswith(key2) )
# # for key,val in model.children:
# #     print(key)

from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("/data/hdj/data/CodeBERT/codesearch/java_model_codesearch/512_0202_del_import_0.62_0.14_0.18/checkpoint-best")

output=None
for i in range(2):
    a=torch.tensor([[1,1,1],[2,2,2]])
    print(a.shape)
    # b=torch.tensor([[3,3,3],[4,4,4]])
    if   output==None:
        output=a.reshape(1,-1)
    else:
        
        output=torch.cat([output,a.view(1,-1)],axis=0)
#     output.append()
output

a=torch.tensor([[1,1,2,2,2]])
output1=torch.cat([a,a],axis=0)
output1

# if hasattr(model, 'module'):
#     print('t')
# else:
#     print('f')
# for key,val in model.state_dict().items():
#     print(key)
# model_to_save = model.module if hasattr(model, 'module') else model
# save_params = {
#         "model": model_to_save.state_dict(),
#         "optimizer_name": 'no_exists',
#         "optimizer": 'optimizer.state_dict()',
#         "trained_epoch": 3,
#         "global_step":1
# }
# torch.save(save_params, '/home/hdj/BERT-PLI-IJCAI2020/output/filenamed.pkl')

filename=torch.load('/home/hdj/BERT-PLI-IJCAI2020/output/filenamed.pkl')
filenameModel=filename['model']


# fine_tune=torch.load('/home/hdj/BERT-PLI-IJCAI2020/output/model/task2/1.pkl')
# fine_tuneModel=fine_tune['model']

# model.state_dict()

# filenameModel

# for key,val in filename['model'].items():
#     print(key)

# for key,val in parameters.items():
#     print(key)


# 模型需要的样例结束

# tomcat_ast_file = pickle.loads(tomcat_ast_cache_collection_db['5d3a94c94ec7de55d2d83b0c8ba4ca4689626309'])
# tomcat_ast_file['methodContent']

# 根据bid 和fid获取相应的字符结束

all_results_df_top_k.index.get_level_values(0).unique().shape[0]
3645/500

all_results_df_top_k.loc[500:999].shape,all_results_df_top_k.loc[1000:1499]

print(all_results_df_top_k.loc[4143].shape)
all_results_df_top_k.loc[4143]

top_k=sorted_df[:50]

# print(top_k.head())
print(top_k.shape)
top_k_pos=top_k['used_in_fix'].sum()
all_pos=sorted_df['used_in_fix'].sum()
print('top_k :',top_k_pos)
print('all :',all_pos)
print('recall :',top_k_pos/all_pos)

# 计算前k个的recall，将前k个文件保存起来结束

sum_val=0
average_precision_per_bug_report = []
reciprocal_ranks = []
k_range=range(1, 21)
bug_report_number = 0
accuracy_at_k = dict.fromkeys(k_range, 0)
for i,(bug_report, bug_report_files_dataframe)  in enumerate(all_results_df_before.groupby(level=0, sort=False)):
    print('index :',i,bug_report,bug_report_files_dataframe.shape[0])
    sum_val+=bug_report_files_dataframe.shape[0]
#     print(bug_report_files_dataframe)
    min_fix_result = bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix'] == 1.0]['result'].min()
    bug_report_files_dataframe2 = bug_report_files_dataframe[bug_report_files_dataframe["result"] >= min_fix_result]
#     print('取fix得分最小值之前的所有文件 ',bug_report_files_dataframe2.shape,' 过滤前大小 :',bug_report_files_dataframe.shape,
#           ' min_fix_result :',min_fix_result)
    sorted_df = bug_report_files_dataframe2.sort_values(ascending=False, by=['result'])
    #假如过滤后没有值，直接用原始的dataframe
    if sorted_df.shape[0] == 0:
        sorted_df = bug_report_files_dataframe.copy().sort_values(ascending=False, by=['result'])
    #为了代码的健壮性
    
    #保存在k位置正例的比例
    precision_at_k = []
    
    # precision per k in range
    #给原始数据增加一列position，使用tmp保存
    tmp = sorted_df
    a = range(1, tmp.shape[0] + 1)
    tmp['position'] = pd.Series(a, index=tmp.index)
#     print(tmp)
    #保存的是fix的排序位置
    large_k_p = tmp[(tmp['used_in_fix'] == 1.0)]['position'].tolist()
#     print('large_k_p :',large_k_p)
    #保存的是result的升序排列
    unique_results = sorted_df['result'].unique().tolist()
    unique_results.sort()
#     print('unique_results :',unique_results)
    #计算在k位置前有多少正例,计算的是MAP的值
    for fk in large_k_p:#[4, 186]
        k = int(fk)
        #取
        k_largest = unique_results[-k:]
        largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
        real_fixes_at_k = (largest_at_k['used_in_fix'] == 1.0).sum()
        #正例个数/位置k
        p = float(real_fixes_at_k) / float(k)
        precision_at_k.append(p)
#     print('precision_at_k :',precision_at_k)
    average_precision = pd.Series(precision_at_k).mean()
    average_precision_per_bug_report.append(average_precision)
#     print('average_precision_per_bug_report :',average_precision_per_bug_report)
    #保存MAP完毕
    
    #计算TOP20
    for k in k_range:
        k_largest = unique_results[-k:]
        largest_at_k = sorted_df[sorted_df['result'] >= min(k_largest)]
        real_fixes_at_k = largest_at_k['used_in_fix'][(largest_at_k['used_in_fix'] == 1.0)].count()
        if real_fixes_at_k >= 1:
            accuracy_at_k[k] += 1
    #计算TOP20完毕
    
    #计算MRR
    indexes_of_fixes = np.flatnonzero(sorted_df['used_in_fix'] == 1.0)
#     print('indexes_of_fixes :',indexes_of_fixes)
    if indexes_of_fixes.size == 0:
        reciprocal_ranks.append(0.0)
    else:
        first_index = indexes_of_fixes[0]
        reciprocal_rank = 1.0 / (first_index + 1)
        reciprocal_ranks.append(reciprocal_rank)
#     print('reciprocal_ranks :',reciprocal_ranks)
    #计算MRR完毕
    bug_report_number += 1
#     print('*'*10)
#     if i==10:
#         break

mean_average_precision = pd.Series(average_precision_per_bug_report).mean()
    # print('mean average precision', mean_average_precision)
mean_reciprocal_rank = pd.Series(reciprocal_ranks).mean()
# print('mean reciprocal rank', mean_reciprocal_rank)
for k in k_range:
    accuracy_at_k[k] = accuracy_at_k[k] / bug_report_number

mean_average_precision,mean_reciprocal_rank,accuracy_at_k

sum([0.25, 0.010752688172043012])/2

# 指标计算结束

all_results_df_before.index.get_level_values(0).unique().shape[0]

fold_test_1 = pd.read_pickle('/data/hdj/tracking_buggy_files/swt/swt_normalized_testing_fold_1')

from skopt import *
fold_number, fold_testing, fold_training = load('/data/hdj/tracking_buggy_files/joblib_memmap_swt/swt/data_memmap', mmap_mode="r")

fold_number,len(fold_testing)

# all_rows=0
# for i in range(1,9):
#     print(fold_testing[i].shape[0])
#     all_rows+=fold_testing[i].shape[0]
# print(all_rows)

# for i in range(500,1000):
#     print(fold_testing[1].loc[i].shape,all_results_df_before.loc[i].shape)

all_results_df_after.shape,type(all_results_df_after)

all_results_df_after.to_csv('/data/hdj/tracking_buggy_files/joblib_memmap_tomcat/after.csv')

all_results_df_after_check=pd.read_csv('/data/hdj/tracking_buggy_files/joblib_memmap_tomcat/after.csv')
all_results_df_after_check.shape

type(queries),type(corpus),len(queries),len(corpus)

# corpus.keys()

# # aspectj_test=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_test.csv')
# all_data=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/all_data.csv')
# all_data.head()
# # aspectj_test[['bug_id','files']].head()

all_fixed_files=[]
for file in all_data['files'].values:
    all_fixed_files.extend(eval(file))

len(all_fixed_files)
# all_fixed_files
with open('/data/hdj/SourceFile/data/sourceFile_aspectj/fixed_files.txt','w',encoding='utf-8') as f_out:
    for file in all_fixed_files:
        f_out.write(file+'\n')

src_addresses = glob.glob(str('/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj') + '/**/*.java', recursive=True)
len(src_addresses)

# aspectj_test_pos_neg['path'][1000],aspectj_test_pos_neg['path'][2]

# aspectj_test_pos_neg=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_test_pos_neg.csv')
# aspectj_test_pos_neg.head(1002)

aspectj_test=pd.read_csv('/data/hdj/SourceFile/data/sourceFile_aspectj/aspectj_test.csv')
# aspectj_test[['bug_id','files']].head()
bug_ids=aspectj_test['bug_id'].values
files=aspectj_test['files'].values
for i,(bid,file) in enumerate(zip(bug_ids,files)):
    print(i,bid,file)
    if i==10:
        break
tests/bugs163/pr251326/pkgA/Target.java

# #测试DictVectorizer()如何使用
# import pandas as pd
# from collections import Counter
# from sklearn.feature_extraction import DictVectorizer
# data=[]
# def convert(dict_list):
#     counter_list = []
#     for d in dict_list:
#         counter_list.append(Counter(d))
#     return counter_list
# test_dict={'a':1,'b':2,'d':3}
# data.extend(convert([test_dict]))

# test_list=[{'a':1,'b':2,'e':3},{'a':1,'b':2,'f':3},{'a':1,'b':2,'g':3}]
# data.extend(convert(test_list))

# vectorizer = DictVectorizer()
# vectorized_data = vectorizer.fit_transform(data)
# feature_names = vectorizer.get_feature_names()
# feature_names_lenghts_dict = {}
# for i, feature_name in enumerate(feature_names):
#     feature_names_lenghts_dict[i] = feature_name
# feature_names_lenghts_dict
# # vectorized_data.toarray()